## Dyalog recipes

To see a correct render of this notebook: https://nbviewer.jupyter.org/github/xpqz/DyalogCookbook/blob/master/Dyalog%20Cookbook.ipynb

This notebook contains my notes from learning APL. A lot of these has come from helpful discussions on the APL Orchard Stack Exchange chatroom.

APL allows you to configure the index origin, which is to say if arrays and vectors start at index 0 or 1. The default is 1, and in this notebook this is what we use, to avoid confusion. If you're used to 0-indexed languages (C, Java, Python etc) this may grate, but in APL it soon feels natural.

In [1]:
⎕IO←1 ⍝ one-based indexing (this is the default, but let's be explicit)
]box on -style=max -trains=tree -fns=on ⍝ Pass all output through DISPLAY

### Regular expressions

Dyalog supports the full PCRE syntax via ⎕S and ⎕R

See http://archive.vector.org.uk/art10500870

In [3]:
⍝ Get all words. The config-string '&' returns the matched string.
'\w+' ⎕S '&' ⊣ 'the cat sat on the mat'

┌→───────────────────────────────────┐
│ ┌→──┐ ┌→──┐ ┌→──┐ ┌→─┐ ┌→──┐ ┌→──┐ │
│ │the│ │cat│ │sat│ │on│ │the│ │mat│ │
│ └───┘ └───┘ └───┘ └──┘ └───┘ └───┘ │
└∊───────────────────────────────────┘

In [4]:
⍝ Reverse each word
'\w+' ⎕S {⌽⍵.Match} ⊣ 'the cat sat on the mat'

┌→───────────────────────────────────┐
│ ┌→──┐ ┌→──┐ ┌→──┐ ┌→─┐ ┌→──┐ ┌→──┐ │
│ │eht│ │tac│ │tas│ │no│ │eht│ │tam│ │
│ └───┘ └───┘ └───┘ └──┘ └───┘ └───┘ │
└∊───────────────────────────────────┘

Note that ⎕S is a dyadic function that returns a function. Left arg is always the pattern. Right arg is either a transformation function or a config string.

In [5]:
⍝ Capture groups
'c([^t]+)t' ⎕S '\1' ⊣ 'The cabriolet parked on the road'

┌→──────────┐
│ ┌→──────┐ │
│ │abriole│ │
│ └───────┘ │
└∊──────────┘

The config string is used to specify the capture group following Perl conventions, \1, \2 etc. To get all captured groups we have to do a little dance:

In [2]:
⊃,/'(hello|goodbye)\s+(\d+)\s+(\d+)'⎕S{⍵.(1↓Lengths↑¨Offsets↓¨⊂Block)} ⊢ 'hello 123 456'

┌─────┬───┬───┐
│hello│123│456│
└─────┴───┴───┘

We can make that into a handy function:

In [6]:
RegexGroups←{⊃,/⍺⎕S{⍵.(1↓Lengths↑¨Offsets↓¨⊂Block)} ⊢ ⍵}

The ⎕R function does a replace:

In [7]:
'[si]'⎕R'S' ⊢ 'mississippi'

┌→──────────┐
│mSSSSSSSppS│
└───────────┘

In [8]:
's'⎕R'S'⍠'ML' 2 ⊢ 'mississippi' ⍝ Set the "match limit" to 2, with the variant operator ⍠

┌→──────────┐
│miSSissippi│
└───────────┘

In [9]:
{⊃('.+' ⎕S '\u0')⍵} 'Hello All People in The World' ⍝ Upper-case a string

┌→────────────────────────────┐
│HELLO ALL PEOPLE IN THE WORLD│
└─────────────────────────────┘

Here's a dfn that takes a regex and a string and splits the latter on the former:

In [10]:
RegSplit←{⊃{⍵/⍨⍺∨¯1⌽⍺}/↓⍉↑('(.*?)',⍺)'(.*?)$'⎕S{⍵.((~PatternNum)(Lengths[2]↑Offsets[2]↓Block))}⊢⍵}
RegSplit2←{(⊢/¨r)↓¨⍵⊂⍨(⍳≢⍵)∊1+⊃¨r←(⍺,'|^')⎕S 0 1⊢⍵} ⍝ faster

In [11]:
'/+' RegSplit 'Here///be/dragons/'  ⍝ Note empty last item in result
'/+' RegSplit2 'Here///be/dragons/'  ⍝ Note empty last item in result

┌→──────────────────────────┐
│ ┌→───┐ ┌→─┐ ┌→──────┐ ┌⊖┐ │
│ │Here│ │be│ │dragons│ │ │ │
│ └────┘ └──┘ └───────┘ └─┘ │
└∊──────────────────────────┘

┌→──────────────────────────┐
│ ┌→───┐ ┌→─┐ ┌→──────┐ ┌⊖┐ │
│ │Here│ │be│ │dragons│ │ │ │
│ └────┘ └──┘ └───────┘ └─┘ │
└∊──────────────────────────┘

Branch reset to use the same sub-pattern name for alternatives

In [3]:
'(?|.*#(\d+).*|.*:(\d\d)).*'⎕r'\1'⊢'[1518-02-15 23:58] Guard #3557 begins shift' '[1518-02-16 00:33] falls asleep'

┌────┬──┐
│3557│33│
└────┴──┘

Overlapping matches in regex -- there is a variant setting (only works for `⎕S`)

In [1]:
'a.'⎕S'&'⍠'OM'1⊢'abaac' ⍝ Variant OM - overlapping matches, nice! 

┌──┬──┬──┐
│ab│aa│ac│
└──┴──┴──┘

or via a zero-width assertion for overlapping capture groups:

In [9]:
'(?=(a.))'⎕S'\1'⊢'abaac'

┌──┬──┬──┐
│ab│aa│ac│
└──┴──┴──┘

## FizzBuzz

FizzBuzz is a well-known task sometimes set at interviews for coding jobs. The idea is to consider a sequence o f numbers and print "FizzBuzz" if the number is divisible by 3 and 5, "Buzz" if divisible by "5", "Fizz" if divisible by 3 and otherwise just output the number.

Here's APL-FizzBuzz, as given in https://rosettacode.org/wiki/FizzBuzz#APL

In [40]:
{⊃⍵ 'Fizz' 'Buzz' 'FizzBuzz'[⎕IO++/1 2×0=3 5|⍵]}¨⍳16

┌→───────────────────────────────────────────────────────────────────────────┐
│ ┌→───┐ ┌→───┐ ┌→───┐ ┌→───┐ ┌→───┐ ┌→───┐ ┌→───────┐ │
│ 1 2 │Fizz│ 4 │Buzz│ │Fizz│ 7 8 │Fizz│ │Buzz│ 11 │Fizz│ 13 14 │FizzBuzz│ 16 │
│ └────┘ └────┘ └────┘ └────┘ └────┘ └────┘ └────────┘ │
└∊───────────────────────────────────────────────────────────────────────────┘

The indexing function `1++/1 2×0=3 5|⍵` generates 1 for numbers not divisible by either 3 or 5, 2 for divisible by 3, 3 for divisible by 5 and 4 for divisible by 3 and 5. So how does that work?

Consider it right to left:

In [16]:
{3 5|⍵}¨10 11 15 ⍝ Return length 2 vector with remainder from div by 3 and 5

┌→──────────────────┐
│ ┌→──┐ ┌→──┐ ┌→──┐ │
│ │1 0│ │2 1│ │0 0│ │
│ └~──┘ └~──┘ └~──┘ │
└∊──────────────────┘

Next step is to pick out the zeros

In [17]:
{0= 3 5|⍵}¨10 11 15 ⍝ Return length 2 vector with remainder from div by 3 and 5, highlighing zeros

┌→──────────────────┐
│ ┌→──┐ ┌→──┐ ┌→──┐ │
│ │0 1│ │0 0│ │1 1│ │
│ └~──┘ └~──┘ └~──┘ │
└∊──────────────────┘

...and multiply by 1 and 2 respectively:

In [18]:
{1 2× 0= 3 5| ⍵}¨10 11 15 ⍝ ...multiplied by 1 and 2

┌→──────────────────┐
│ ┌→──┐ ┌→──┐ ┌→──┐ │
│ │0 2│ │0 0│ │1 2│ │
│ └~──┘ └~──┘ └~──┘ │
└∊──────────────────┘

Now what remains is to sum that up, and add `⎕IO`:

In [41]:
{⎕IO++/1 2× 0= 3 5|⍵}¨10 11 15 ⍝ ...multiplied by 1 and 2, and summed

┌→────┐
│3 1 4│
└~────┘

We can now index into the vector containing our output text, after disclosing:

In [42]:
{⊃⍵ 'Fizz' 'Buzz' 'FizzBuzz'[⎕IO++/ 1 2× 0= 3 5| ⍵]} ¨ 3 10 11 15

┌→────────────────────────────┐
│ ┌→───┐ ┌→───┐ ┌→───────┐ │
│ │Fizz│ │Buzz│ 11 │FizzBuzz│ │
│ └────┘ └────┘ └────────┘ │
└∊────────────────────────────┘

This version is iterative, as it uses each (`¨`). We can take a radically different approach to take advantage of APL's array operators:

In [5]:
]dinput
FizzBuzz←{
    nums←⍳⍵
    mat←(⍱⌿⍪⊢)0=3 5∘.|nums  ⍝ Remainder matrix
    mat×@1⍨←nums            ⍝ Multiply first row by the source numbers
    mat←(⊂'Fizz')@⊢@2⊢mat   ⍝ Replace any 1s in second row by Fizz
    mat←(⊂'Buzz')@⊢@3⊢mat   ⍝ Replace any 1s in third row by Buzz
    0~⍨¨,⌿mat               ⍝ Merge downwards and remove any zeros
}

In [6]:
FizzBuzz 16

┌→─────────────────────────────────────────────────────────────────────────────────────────────┐
│ ┌→┐ ┌→┐ ┌→───┐ ┌→┐ ┌→───┐ ┌→───┐ ┌→┐ ┌→┐ ┌→───┐ ┌→───┐ ┌→─┐ ┌→───┐ ┌→─┐ ┌→─┐ ┌→───────┐ ┌→─┐ │
│ │1│ │2│ │Fizz│ │4│ │Buzz│ │Fizz│ │7│ │8│ │Fizz│ │Buzz│ │11│ │Fizz│ │13│ │14│ │FizzBuzz│ │16│ │
│ └~┘ └~┘ └────┘ └~┘ └────┘ └────┘ └~┘ └~┘ └────┘ └────┘ └~─┘ └────┘ └~─┘ └~─┘ └────────┘ └~─┘ │
└∊─────────────────────────────────────────────────────────────────────────────────────────────┘

### Split a string into a vector of a single char and a number

E.g 'X1234' gives 'X' 1234 -- via @Adám on APLOrchard

In [35]:
1(↑,∘⍎↓)'X1234' ⍝ Drop the first char, evaluate the rest, and catenate the first char.

┌→─────┐
│X 1234│
└+─────┘

Of course, using ⍎ on unfiltered user input can be dangerous. 

In [36]:
(⊃,∘⍎∩∘⎕D)'X4234' ⍝ safer, but can throw an error.

┌→─────┐
│X 4234│
└+─────┘

Here the [⎕D](http://help.dyalog.com/latest/#Language/System%20Functions/d.htm) is a system variable specifying "digits":

In [37]:
⎕D

┌→─────────┐
│0123456789│
└──────────┘

The industrial strength version uses [⎕VFI](http://help.dyalog.com/latest/#Language/System%20Functions/vfi.htm), (Verify and Fix Input) to properly -- and safely -- parse the integer part.

In [45]:
1(↑,2⊃∘⎕VFI↓)'X1234' 

┌→─────┐
│X 1234│
└+─────┘

Verify and Fix Input, when used dyadically, can be used to split and convert strings:

In [47]:
2⊃'x'⎕VFI'19x29x21'

┌→───────┐
│19 29 21│
└~───────┘

We can see how the first version is derived from a perhaps more obvious starting point:

In [48]:
{a←1↑⍵ ⋄ b←⍎1↓⍵ ⋄ a,b} 'X1234' ⍝ Take head, eval tail

{(1↑⍵),(⍎1↓⍵)} 'X1234'         ⍝ substitute variables
1 {(⍺↑⍵),(⍎⍺↓⍵)} 'X1234'       ⍝ break out 1
1 ((⊣↑⊢),(⍎⊣↓⊢)) 'X1234'       ⍝ train
1 ((↑),(⍎↓)) 'X1234'           ⍝ simplify
1 (↑,∘⍎↓) 'X1234'              ⍝ remove parens

┌→─────┐
│X 1234│
└+─────┘

┌→─────┐
│X 1234│
└+─────┘

┌→─────┐
│X 1234│
└+─────┘

┌→─────┐
│X 1234│
└+─────┘

┌→─────┐
│X 1234│
└+─────┘

┌→─────┐
│X 1234│
└+─────┘

### Partition a vector into groups of equal elements

Good partition primer on [APLWiki](https://aplwiki.com/wiki/Partition_representations)

In [51]:
partition←{⍵⊂⍨1,2≠/⍵}
partition 1 1 1 1 1 2 2 1 4 4 4 4 1 1 2 2

┌→────────────────────────────────────────────┐
│ ┌→────────┐ ┌→──┐ ┌→┐ ┌→──────┐ ┌→──┐ ┌→──┐ │
│ │1 1 1 1 1│ │2 2│ │1│ │4 4 4 4│ │1 1│ │2 2│ │
│ └~────────┘ └~──┘ └~┘ └~──────┘ └~──┘ └~──┘ │
└∊────────────────────────────────────────────┘

If we need the run lengths, we can just count them up:

In [52]:
≢¨partition 1 1 1 1 1 2 2 1 4 4 4 4 1 1 2 2

┌→──────────┐
│5 2 1 4 2 2│
└~──────────┘

### Match brackets

In [58]:
{+\1 ¯1 0['()'⍳⍵]} '((2×3)+4)' 

┌→────────────────┐
│1 2 2 2 2 1 1 1 0│
└~────────────────┘

Nice! That snippet was supposedly penned by Ken himself. 

https://forums.dyalog.com/viewtopic.php?f=30&t=1616
https://www.jsoftware.com/papers/perlis78.htm

How does it work?

Well, the expression `'()'⍳⍵` creates a vector of the same length as the argument containing a 1 (or 0, depending on `⎕io`) if the corresponding character is a '(', 2 for ')' and 3 (overflow) for everything else. We then map those values to 1, ¯1 and 0 respectively, and scan-sum over those.

Another way, slightly shorter, to achieve the same thing:

In [59]:
{+\-⌿'()'∘.=⍵} '((2×3)+4)'  ⍝ Match brackets

┌→────────────────┐
│1 2 2 2 2 1 1 1 0│
└~────────────────┘

This version builds a matrix where the two rows are binary masks showing the positions of the respective brackets.

In [60]:
'()'∘.='((2×3)+4)'

┌→────────────────┐
↓1 1 0 0 0 0 0 0 0│
│0 0 0 0 0 1 0 0 1│
└~────────────────┘

Next step is to subtract row 2 from row 1

In [61]:
-⌿'()'∘.='((2×3)+4)'

┌→──────────────────┐
│1 1 0 0 0 ¯1 0 0 ¯1│
└~──────────────────┘

and then scan-sum

In [62]:
+\-⌿'()'∘.='((2×3)+4)'

┌→────────────────┐
│1 2 2 2 2 1 1 1 0│
└~────────────────┘

## Stdlib

In [3]:
⍝ Zips -- remix is basically zip 
Zip←↓⍉↑
PythonZip←↓∘⍉∘↑(⌊⌿≢¨)↑¨⊢  ⍝ Python's behaviour is to only zip to shortest vector if unmatched
PythonZip2←↓(⌊/≢¨)↑⍉∘↑

In [ ]:
NotUnzip←|∘⍳∘≢⊢∘⊂⌸⊢                                    ⍝ 2 Unzip 'dyalog' NB: Not really unzip as such

In [ ]:
Split←≠⊆⊢                                           ⍝ Split string on a char

In [13]:
MSplit←(~∊⍨)⊆⊢                                      ⍝ Multi-char split

In [13]:
RegSplit←{(⊢/¨r)↓¨⍵⊂⍨(⍳≢⍵)∊1+⊃¨r←(⍺,'|^')⎕S 0 1⊢⍵}  ⍝ Split string on a pattern

In [ ]:
RegexGroups←{⍺⎕S{⍵.(1↓Lengths↑¨Offsets↓¨⊂Block)} ⊢ ⍵} ⍝ Return all capture groups as a vector

In [1]:
Range←⊣+∘⍳-⍨                                        ⍝ start Range end, but see dfns.iotag instead
iota←{⍺+0,+\(|d)⍴×d←⍵-⍺}                            ⍝ Guts of iotag, works for negative ⍺ and ⍵
iota_io1←{⍺+¯1+⍳1+⍵-⍺}

Rebuilding user command cache... done

Rebuilding user command cache... done

In [ ]:
Replace←' '@(=∘'-')⊢

In [ ]:
Foldl←{↑⍺⍺⍨/(⌽⍵),⊂⍺}                                ⍝ 0 + Foldl 1 2 3 4 5 ⍝ Must give initial accumulator state

In [ ]:
Flatten←{⊃,/,⊆¨⍵}⍣≡

In [ ]:
Filter←{⍵/⍨⍺⍺ ⍵}                                    ⍝ Simple list only: {⍵<10} Filter 1 2 55 27 11 6 5 0

In [ ]:
FilterNested←{⍵/⍨⍺⍺¨⍵}                             

In [ ]:
Pairs←{,⍳⍵ ⍵}

In [ ]:
OverlappingPairs←{{⊂⍵}⌺2 ⊢ ⍵}

In [15]:
Balance←{+\-⌿⍺∘.=⍵}                                 ⍝ '()' Balance '(a+b+(1-c))'

In [ ]:
Normalise←⊢÷+/                                      ⍝ Make vector components sum to 1

In [ ]:
Divisors←{⍵=1:,1⋄∊∘.×/{⍺∘.*0,⍳⍵}⌿2 dfns.pco ⍵}

In [ ]:
DivisiorSum←{×/{(¯1+⍺*⍵+1)÷⍺-1}⌿2 dfns.pco ⍵}       ⍝ A.k.a sigma1

In [ ]:
SortByCol←{⍵[⍋⍵[;⍺];]}                              ⍝ col SortByCol mat

In [1]:
Convert←{6::⍵⋄⍎⍵}                                   ⍝ Note -- unsafe
conv←{d←⎕VFI ⍵⋄0=⊃⊃d:⍵⋄⊃⊃⊖d}                        ⍝ better

Rebuilding user command cache... 
done

In [1]:
]dinput
Cmb←{                                               ⍝ Find all combinations of N numbers ∊⍳M that sum to M
    (num total)←⍵
    A←⊃∘.+/(num-1)⍴⊂⍳total
    ⊃,/{⍵,¨⍸(total-⍵)=A}¨⍳total
}

In [2]:
ConvToIdx←∪⍳⊢                                       ⍝ convert strings to IDs

In [ ]:
ConvToDigits←10∘⊥⍣¯1                                ⍝ Split a number into a vector of digits

In [3]:
IsNumber←{(1=2|⎕DR)⍵}                               ⍝ Are we a number/numeric?
IsString←⍕≡⊢                                        ⍝ Are we a simple character vector?
IsNamespace←{(326=⎕DR⍵)}                            ⍝ Are we a reference/object/namespace?

In [ ]:
Substring←⍸⍷                                        ⍝ 'll' (⍸⍷) 'hello' ⍝ 3

In [ ]:
AllSubstr←⍳∘≢,/¨⊂                                   ⍝ All possible substrings grouped by length

In [ ]:
Del←{⍵/⍨0@⍺⊢1⍨¨⍵}                                   ⍝ 2 5 Del ⍳10 ⍝ remove indices ⍺ from array ⍵

In [ ]:
Group←↓⊃¨,∘⊂⌸⊢/¨                                    ⍝ Group vector elements based on first

In [ ]:
GroupSplit←{⍵⊆⍨×≢¨⍵}                                ⍝ Group vector elements separated by empty

In [ ]:
BinaryCombos←{⍉(⍵⍴2)⊤⍳2*⍵}                          ⍝ All binary combinations of ⍵ bits
FasterBinaryCombos←{⍉2∘⊥⍣¯1⍳2*⍵}

In [12]:
ExtractNumbers←⍎¨∊∘⎕d⊆⊢

In [1]:
MostFrequentWord←{⎕IO←1⋄⍵⌷⍨⊂0~⍨⊢/0,⊢⌸⍵}             ⍝ MostFrequentWord 'hello' 'world' 'hello' 'the' 'other' 'world' 'hello'

In [2]:
HeadTail←1 1∘⊂                                      ⍝ Split a vector into head and tail

### Error handling

Using error guards

In [63]:
4 {3::'out of bounds' ⋄ ⍺ ⌷ ⍵} 1 2 3 4 5 6 7

4

In [64]:
8 {3::'out of bounds' ⋄ ⍺ ⌷ ⍵} 1 2 3 4 5 6 7

┌→────────────┐
│out of bounds│
└─────────────┘

Note that the body of an error guard is evaluated _after_ the local environment is unwound. This is likely not what you expect if you come from say Python:

In [5]:
]dinput
F1←{ ⍝ Can you spot the bug?
    11::nums
    nums←1 ⍺
    nums×÷⍵
}

In [6]:
]dinput
F2←{
    nums←1 ⍺
    0=⍵:nums
    nums×÷⍵
}

In [7]:
4 F2 0 ⍝ The expected behaviour

1 4

In [10]:
4 F1 0 ⍝ Throws a perhaps unexpected VALUE ERROR

VALUE ERROR: Undefined name: nums
F1[1] 11::nums
          ∧


Further, the mere presence of an error guard _disables tail call optimisation_. 

### Sets

Set XOR - union minus intersection, using a train

In [66]:
⊃(∪~∩)/ (1 2 3 4 5 6) (4 5 6 7 8) ⍝ Elements in ⍺ or ⍵ but not in both

┌→────────┐
│1 2 3 7 8│
└~────────┘

### Encode and decode ⊤ ⊥

Encode and decode converts between decimal and some encoding vector.

Split a number into digits and pad with zeros from the left:

In [67]:
10 10 10 10 10 ⊤ 12

┌→────────┐
│0 0 0 1 2│
└~────────┘

Convert from seconds to H M S:

In [68]:
0 24 60 60 ⊤ 8473

┌→────────┐
│0 2 21 13│
└~────────┘

Decode goes the other way

In [69]:
0 24 60 60 ⊥ 0 2 21 13 

8473

Sometimes swapped to save some brackets

In [70]:
12 ⊤⍨ 5⍴10 ⍝ instead of (5⍴10) ⊤ 12

┌→────────┐
│0 0 0 1 2│
└~────────┘

We can exploit decode to sum elements of a vector, which faster than the normal +/ approach and a useful trick in trains.

In [71]:
1⊥1 3 2 5 6

17

## Trains

Trains are APL's functional composition mechanism.

Here are the rules for 2 and 3-carriage trains:

```
(F G H) x -> (F x) G (H x)
(u G H) x -> u G (H x)
(G H) x -> G (H x)

x (F G H) y -> (x F y) G (x H y)
x (u G H) y -> u G (x H y)
x (G H) y -> G (x H y)
```

In [55]:
⍝ Construct a vector of the sum and product of two numbers
2 (+,×) 5  ⍝ x (F G H) y -> (x F y) G (x H y)

┌→───┐
│7 10│
└~───┘

This is a three-function train, consisting of sum, catenate, product. For a three-function train (LMR) in the dyadic case, the execution flow is (⍺ L ⍵) M (⍺ R ⍵) or specifically (2+5),(2×5).

Here's another example: split a string on a separator:

In [73]:
',' (≠⊆⊢) 'one,two,three'

┌→────────────────────┐
│ ┌→──┐ ┌→──┐ ┌→────┐ │
│ │one│ │two│ │three│ │
│ └───┘ └───┘ └─────┘ │
└∊────────────────────┘

Given the execution flow above we have:

In [74]:
Lhs ← ','≠'one,two,three'  ⍝ Binary match vector
Rhs ← ','⊢'one,two,three'  ⍝ Just return the right-hand argument
Lhs ⊆ Rhs                  ⍝ Partition based on binary match vector

┌→────────────────────┐
│ ┌→──┐ ┌→──┐ ┌→────┐ │
│ │one│ │two│ │three│ │
│ └───┘ └───┘ └─────┘ │
└∊────────────────────┘

The right-tack ⊢ is used in trains as an identity function to force the correct monadic/dyadic behaviour.

Another canonical example is calculating the mean with a three-function monadic train:

In [75]:
(+/÷≢) 3 6 4 9

5.5

This translates to:

In [76]:
Lhs ← +/ 3 6 4 9   ⍝ Sum 
Rhs ← ≢ 3 6 4 9    ⍝ Count
Lhs ÷ Rhs          ⍝ Mean

5.5

Set XOR using a union-not-intersect train

In [77]:
⊃(∪~∩)/ (1 2 3 4 5 6) (4 5 6 7 8)

┌→────────┐
│1 2 3 7 8│
└~────────┘

Here's a longer example: generate an integer range

In [79]:
¯1+5 (⊣+∘⍳-⍨) 10

┌→────────┐
│5 6 7 8 9│
└~────────┘

and an even longer, splitting a vector in n parts by unzipping:

In [80]:
2 (|∘⍳∘≢⊢∘⊂⌸⊢) 'dyaloge'

┌→─────────────┐
│ ┌→───┐ ┌→──┐ │
│ │daoe│ │ylg│ │
│ └────┘ └───┘ │
└∊─────────────┘

The following tactit groups pairs based on the first component of each pair:

In [1]:
m←(5 3)(5 6)(7 5)(4 7)(1 8)(2 4)(1 2)
(↓⊃¨,∘⊂⌸⊢/¨)m

Rebuilding user command cache... done

┌───────┬─────┬─────┬───────┬─────┐
│┌─┬───┐│┌─┬─┐│┌─┬─┐│┌─┬───┐│┌─┬─┐│
││5│3 6│││7│5│││4│7│││1│8 2│││2│4││
│└─┴───┘│└─┴─┘│└─┴─┘│└─┴───┘│└─┴─┘│
└───────┴─────┴─────┴───────┴─────┘

Let's unwind that one in steps. Step 1: separate the tacit expression into individual functions, taking into account operators:

    (↓) (⊃¨) (,∘⊂⌸) (⊢/¨)
    
Step 2: convert to dfn by considering first any 3-trains from the right:

    { (⊃¯⍵) (,∘⊂⌸) ⊢/¨⍵ }
    
The middle bit is idiomatic use of key that can also be written as `{⍺⍵}⌸`. So we're looking at a left argument which is simply a vector of all first components of the pairs, and a right argument which is the last component of every pair, applied to the dyadic key, which groups the right arg based on the unique elements in the left.

Step 3: as we've only a remaining `↓`, it's simply applied to the result.

## Tack tricks

Left (⊣) and right (⊢) tacks return the argument they point to.

In [81]:
'Left'⊣'Right' ⋄ 'Left'⊢'Right'

┌→───┐
│Left│
└────┘
┌→────┐
│Right│
└─────┘

We saw earlier their place in trains to refer to left or right arguments.

In [82]:
',' (≠⊆⊢) 'one,two,three'

┌→────────────────────┐
│ ┌→──┐ ┌→──┐ ┌→────┐ │
│ │one│ │two│ │three│ │
│ └───┘ └───┘ └─────┘ │
└∊────────────────────┘

Another use is when we want to return a value after first mutating it.

In [83]:
mem ← 1 2 3 4 5
mem ⊣ mem[1 2] ← 10 10

┌→──────────┐
│10 10 3 4 5│
└~──────────┘

We can also use tacks to pick the first or last column of a matrix. This is a Dyalog [idiom](https://help.dyalog.com/17.1/#Language/Defined%20Functions%20and%20Operators/Idiom%20Recognition/Idiom%20List.htm)
meaning it's highly optimised.

In [84]:
⎕←m←2 3⍴1 2 3 4 5 6 7 8 9
⎕←first←⊣/m
⎕←last←⊢/m

┌→────┐
↓1 2 3│
│4 5 6│
└~────┘

┌→──┐
│1 4│
└~──┘

┌→──┐
│3 6│
└~──┘

## Indexing

There are several ways of indexing into arrays and vectors. Crucially, elements of vectors and matrices are always scalars, but a scalar can be a boxed-up vector or matrix.

Indexing with [] or ⌷ returns the box, not the element, although if the element is a simple scalar, it's the same thing.

In [85]:
v←⍳10
v[5]
v[5 2]
3⌷v

5

┌→──┐
│5 2│
└~──┘

3

In [86]:
v←(1 2 3)(4 5 6)(7 8 9)
v[1]  ⍝ Return the box at element 1

┌─────────┐
│ ┌→────┐ │
│ │1 2 3│ │
│ └~────┘ │
└∊────────┘

To get to the boxed element, we need to either disclose, or pick:

In [87]:
⊃v[1]  ⍝ Open box (disclose)
1⊃v    ⍝ Pick element at 1

┌→────┐
│1 2 3│
└~────┘

┌→────┐
│1 2 3│
└~────┘

The same box-unbox rules also apply to mutation:

In [90]:
v[2]←1 2 3 ⍝ Will fail with LENGTH ERROR, as value isn't boxed.

LENGTH ERROR
      v[2]←1 2 3 ⍝ Will fail with LENGTH ERROR, as value isn't boxed.
          ∧


Instead we need to explicitly enclose the new vector:

In [92]:
v[2]←⊂1 2 3
v

┌→────────────────────────┐
│ ┌→────┐ ┌→────┐ ┌→────┐ │
│ │1 2 3│ │1 2 3│ │7 8 9│ │
│ └~────┘ └~────┘ └~────┘ │
└∊────────────────────────┘

### Set values with @
The `@` operator is very powerful and can be applied in several ways. In the simples case, we can give a list of values and a list of indexes where to set them. This creates a new copy of the right argument, with the new values.

See https://stackoverflow.com/questions/64259704/apl-arrays-element-replacement-and-multiplication

In [97]:
9 9 @ 3 4 ⊢ vec←1 2 3 4 5 6 7 8             ⍝ Note -- no mutation
vec 

┌→──────────────┐
│1 2 9 9 5 6 7 8│
└~──────────────┘

┌→──────────────┐
│1 2 3 4 5 6 7 8│
└~──────────────┘

Contrast this to direct assignment, which mutates in place:

In [98]:
vec ⊣ vec[4 5] ← 8 8          ⍝ Note -- direct assignment mutates v

┌→──────────────┐
│1 2 3 8 8 6 7 8│
└~──────────────┘

The @ operator can also apply functions which modify the values in several different ways. The monadic case

In [99]:
(-@2 5)10 20 30 40 50 60  ⍝ Apply monadic - at positions 2 and 5

┌→──────────────────┐
│10 ¯20 30 40 ¯50 60│
└~──────────────────┘

Similarly, we can apply dyads:

In [100]:
7(+@2 5)10 20 30 40 50 60 

┌→────────────────┐
│10 27 30 40 57 60│
└~────────────────┘

In [ ]:
'x'@(∊∘⎕A)'Hello World' ⍝ Replace all uppercase letters with 'x'. ⎕A gives the uppercase letters.

We can also apply `@` with a function that returns a binary vector when applied to the right argument of the derived function, e.g:

In [17]:
'*'@(2∘|) 1 2 3 4 5     ⍝ Replace odds with '*'

┌→────────┐
│* 2 * 4 *│
└+────────┘

or as simple as

In [19]:
'*'@{1 0 1 0 1}1 2 3 4 5

┌→────────┐
│* 2 * 4 *│
└+────────┘

For a more involved example, let's write an expression that given a vector of values and a vector of row indices appends values to a column in the argument matrix:

In [33]:
mat←↑{(⍵ 1) (0 0)}¨3+⍳5  ⍝ Start state: a 5x2 matrix, columns are vectors
vals←⍳5                ⍝ Values to append to the first column
rows←?⍨5               ⍝ ...of these rows

In [49]:
(,¨∘vals)@(rows,¨1)⊢mat

┌→──────────────┐
↓ ┌→────┐ ┌→──┐ │
│ │4 1 5│ │0 0│ │
│ └~────┘ └~──┘ │
│ ┌→────┐ ┌→──┐ │
│ │5 1 1│ │0 0│ │
│ └~────┘ └~──┘ │
│ ┌→────┐ ┌→──┐ │
│ │6 1 3│ │0 0│ │
│ └~────┘ └~──┘ │
│ ┌→────┐ ┌→──┐ │
│ │7 1 4│ │0 0│ │
│ └~────┘ └~──┘ │
│ ┌→────┐ ┌→──┐ │
│ │8 1 2│ │0 0│ │
│ └~────┘ └~──┘ │
└∊──────────────┘

How does this work? The right arg of `@` fleshes out the list of row indices to include the column we target:

In [35]:
rows
rows,¨1

┌→────────┐
│2 5 3 4 1│
└~────────┘

┌→──────────────────────────────┐
│ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ │
│ │2 1│ │5 1│ │3 1│ │4 1│ │1 1│ │
│ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ │
└∊──────────────────────────────┘

This gives us a complete set of row-col pairs. Let's look at the tacit left argument. It's parsed like so

```
   ∘               
  ┌┴┐              
  ¨ vals
┌─┘                
,                  
```
which is just `{⍵,¨vals}` as an explicit dfn.

In [36]:
{⍵,¨vals} mat[rows,¨1]

┌→────────────────────────────────────────┐
│ ┌→────┐ ┌→────┐ ┌→────┐ ┌→────┐ ┌→────┐ │
│ │5 1 1│ │8 1 2│ │6 1 3│ │7 1 4│ │4 1 5│ │
│ └~────┘ └~────┘ └~────┘ └~────┘ └~────┘ │
└∊────────────────────────────────────────┘

Finally, `@` inserts everything back.

Let's look at some types of 'search & replace'. Given an array, some specific values and corresponding replacement values

In [4]:
array←?10 3⍴10
specific←1 2 3 4
replacements←0 10 100 1000

In [5]:
{replacements[specific⍳⍵]}@{⍵∊specific} array

1000 100 1000
 10 0 5
 100 10 10
 10 10 5
 9 8 10
1000 10 10
 6 8 10
 10 0 10
 9 10 10
 9 9 10

So this is the `(f@g)Y` form of `@`. See all forms in [APLCart](https://aplcart.info/?q=%40%20primitive#). The right-hand function should return a binary map indicating which positions in the argument array are to be changed. The left-hand function is given each corresponding value as `⍵`, and should return the new value.

An alternative formulation mirroring the way that `⎕R` works is:

In [7]:
Repl←{⍺⍺(⍵⍵⌷⍨∘⊂⍳)@(∊∘⍺⍺)⍵}
specific Repl replacements ⊢ array

1000 100 1000
 10 0 5
 100 10 10
 10 10 5
 9 8 10
1000 10 10
 6 8 10
 10 0 10
 9 10 10
 9 9 10

See https://aplcart.info/?q=search-and-replace#

We can pick columns any number of ways:

In [21]:
⎕←mat←3 4⍴?⍳12

┌→───────┐
↓1 2 1 2│
│2 1 6 2│
│9 9 10 1│
└~───────┘

In [29]:
2↑⍤1⊢mat        ⍝ Two first, as proper columns
mat[;1 2]       ⍝ Brackets, as proper columns
(⊂1 2)⌷[2]mat   ⍝ Using squad ⌷, somewhat clumsily

┌→──┐
↓1 2│
│2 1│
│9 9│
└~──┘

┌→──┐
↓1 2│
│2 1│
│9 9│
└~──┘

┌→──┐
↓1 2│
│2 1│
│9 9│
└~──┘

First location of element:

In [27]:
keys←(0 0)(0 1)(3 2)
keys ⍳ (⊂0 1)

2

Bind argument to a lookup function for an immutable "hash table":

In [15]:
find←keys∘⍳
find ⊂0 9

4

## Products

Cartesian product

Think of the arguments to iota as defining the shape of the result:

In [106]:
pairs ← {,⍳⍵ ⍵}
pairs 3

┌→──────────────────────────────────────────────────────┐
│ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ │
│ │1 1│ │1 2│ │1 3│ │2 1│ │2 2│ │2 3│ │3 1│ │3 2│ │3 3│ │
│ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ │
└∊──────────────────────────────────────────────────────┘

To pair the elements of two equal-length vectors, we can do:

In [50]:
1 2 3 4,¨5 6 7 8

┌→────────────────────────┐
│ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ │
│ │1 5│ │2 6│ │3 7│ │4 8│ │
│ └~──┘ └~──┘ └~──┘ └~──┘ │
└∊────────────────────────┘

or

In [51]:
↓⍉↑(1 2 3 4)(5 6 7 8)

┌→────────────────────────┐
│ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ │
│ │1 5│ │2 6│ │3 7│ │4 8│ │
│ └~──┘ └~──┘ └~──┘ └~──┘ │
└∊────────────────────────┘

Primes between 1 and 100, but see fast prime sieve `pco` in `dfns`.

In [107]:
(~R∊R∘.×R)/R←1↓⍳100

┌→─────────────────────────────────────────────────────────────────────┐
│2 3 5 7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89 97│
└~─────────────────────────────────────────────────────────────────────┘

## Composition (Currying)

We can curry a dyadic function down to a monadic function by fixing either left or right argument:

In [108]:
sum←{⍺+⍵}
add5←5∘sum

In [109]:
add5 7

12

In [110]:
add5 1 2 3 4 5 6 7

┌→───────────────┐
│6 7 8 9 10 11 12│
└~───────────────┘

## Shape and Rank

Pair consecutive elements in a vector

In [111]:
{⊂⍵}⌺(⍪2 2) ⊢ ⍳10

┌→───────────────────────────────┐
│ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→───┐ │
│ │1 2│ │3 4│ │5 6│ │7 8│ │9 10│ │
│ └~──┘ └~──┘ └~──┘ └~──┘ └~───┘ │
└∊───────────────────────────────┘

In [112]:
,⌿⍉5 2 ⍴ ⍳10

┌→───────────────────────────────┐
│ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→───┐ │
│ │1 2│ │3 4│ │5 6│ │7 8│ │9 10│ │
│ └~──┘ └~──┘ └~──┘ └~──┘ └~───┘ │
└∊───────────────────────────────┘

In [113]:
↓5 2 ⍴ ⍳10

┌→───────────────────────────────┐
│ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→───┐ │
│ │1 2│ │3 4│ │5 6│ │7 8│ │9 10│ │
│ └~──┘ └~──┘ └~──┘ └~──┘ └~───┘ │
└∊───────────────────────────────┘

Probably most idiomatic: windowed reduction

In [16]:
2,⌿⍳10

┌───┬───┬───┬───┬───┬───┬───┬───┬───┐
│0 1│1 2│2 3│3 4│4 5│5 6│6 7│7 8│8 9│
└───┴───┴───┴───┴───┴───┴───┴───┴───┘

Overlapping pairs

In [54]:
{⊂⍵}⌺2 ⊢ ⍳10

┌→───────────────────────────────────────────────────────┐
│ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→───┐ │
│ │1 2│ │2 3│ │3 4│ │4 5│ │5 6│ │6 7│ │7 8│ │8 9│ │9 10│ │
│ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~───┘ │
└∊───────────────────────────────────────────────────────┘

## Folds

The reduce operator / folds R-L. It's possible to define a fold operator that works L-R. All this does is to prepend the accumulator, and reverse the array we're folding before applying the normal fold.

In [115]:
0 + foldl 1 2 3 4 5 ⍝ Must give initial accumulator state

15

Note that this operator (and many others) exists in the standard Dyalog workspace [dfns](https://dfns.dyalog.com/s_foldl.htm), which can be imported as ⎕CY 'dfns'

## On ranges

In Python, for example, you can specify a [range](https://docs.python.org/3/library/functions.html#func-range) as `range(start, end)` which gives an iterator from start to end-1. In Dyalog there isn't a direct equivalent, but one can be made from the iota operator.

In [116]:
¯1+5 {⍺↓⍳⍵} 10   ⍝ Wasteful, as generating from ⎕IO

┌→────────┐
│5 6 7 8 9│
└~────────┘

In [117]:
¯1+5 (⊣↓∘⍳) 10   ⍝ The abobve as a train

┌→────────┐
│5 6 7 8 9│
└~────────┘

In [118]:
¯1+5 {⍺+⍳⍵-⍺} 10 ⍝ Better!

┌→────────┐
│5 6 7 8 9│
└~────────┘

In [119]:
¯1+5 (⊣+∘⍳-⍨) 10 ⍝ The above as a train

┌→────────┐
│5 6 7 8 9│
└~────────┘

In [1]:
'iotag'⎕CY'dfns' ⍝ iotag -- generalised iota
12 iotag 3 3 ⍝ 12 downto 3, step 3

Rebuilding user command cache... done

12 9 6 3

## Stencils

Pick out 3x3 regions of a larger matrix. The stencil takes a function to the left and a shape to the right and returns a monad.

This from https://chat.stackexchange.com/rooms/52405/conversation/lesson-5-even-more-apl-operators--

In [120]:
⎕←letters ← 4 6⍴⎕A
{⊂⍵}⌺3 3 ⊢ letters     ⍝ Enclose the selected window

┌→─────┐
↓ABCDEF│
│GHIJKL│
│MNOPQR│
│STUVWX│
└──────┘

┌→────────────────────────────────────┐
↓ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ │
│ ↓ │ ↓ │ ↓ │ ↓ │ ↓ │ ↓ │ │
│ │ AB│ │ABC│ │BCD│ │CDE│ │DEF│ │EF │ │
│ │ GH│ │GHI│ │HIJ│ │IJK│ │JKL│ │KL │ │
│ └───┘ └───┘ └───┘ └───┘ └───┘ └───┘ │
│ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ │
│ ↓ AB│ ↓ABC│ ↓BCD│ ↓CDE│ ↓DEF│ ↓EF │ │
│ │ GH│ │GHI│ │HIJ│ │IJK│ │JKL│ │KL │ │
│ │ MN│ │MNO│ │NOP│ │OPQ│ │PQR│ │QR │ │
│ └───┘ └───┘ └───┘ └───┘ └───┘ └───┘ │
│ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ │
│ ↓ GH│ ↓GHI│ ↓HIJ│ ↓IJK│ ↓JKL│ ↓KL │ │
│ │ MN│ │MNO│ │NOP│ │OPQ│ │PQR│ │QR │ │
│ │ ST│ │STU│ │TUV│ │UVW│ │VWX│ │WX │ │
│ └───┘ └───┘ └───┘ └───┘ └───┘ └───┘ │
│ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ │
│ ↓ MN│ ↓MNO│ ↓NOP│ ↓OPQ│ ↓PQR│ ↓QR │ │
│ │ ST│ │STU│ │TUV│ │UVW│ │VWX│ │WX │ │
│ │ │ │ │ │ │ │ │ │ │ │ │ │
│ └───┘ └───┘ └───┘ └───┘ └───┘ └───┘ │
└∊────────────────────────────────────┘

The padding is defined by ⍺ in the operand function:

In [121]:
({⊂⍺}⌺3 3) ⊢ letters

┌→───────────────────────────────────────────┐
↓ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→───┐ │
│ │1 1│ │1 0│ │1 0│ │1 0│ │1 0│ │1 ¯1│ │
│ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~───┘ │
│ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→───┐ │
│ │0 1│ │0 0│ │0 0│ │0 0│ │0 0│ │0 ¯1│ │
│ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~───┘ │
│ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→───┐ │
│ │0 1│ │0 0│ │0 0│ │0 0│ │0 0│ │0 ¯1│ │
│ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~───┘ │
│ ┌→───┐ ┌→───┐ ┌→───┐ ┌→───┐ ┌→───┐ ┌→────┐ │
│ │¯1 1│ │¯1 0│ │¯1 0│ │¯1 0│ │¯1 0│ │¯1 ¯1│ │
│ └~───┘ └~───┘ └~───┘ └~───┘ └~───┘ └~────┘ │
└∊───────────────────────────────────────────┘

In [122]:
({⊂⍺↓⍵}⌺3 3) ⊢ letters

┌→──────────────────────────────────┐
↓ ┌→─┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→─┐ │
│ ↓AB│ ↓ABC│ ↓BCD│ ↓CDE│ ↓DEF│ ↓EF│ │
│ │GH│ │GHI│ │HIJ│ │IJK│ │JKL│ │KL│ │
│ └──┘ └───┘ └───┘ └───┘ └───┘ └──┘ │
│ ┌→─┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→─┐ │
│ ↓AB│ ↓ABC│ ↓BCD│ ↓CDE│ ↓DEF│ ↓EF│ │
│ │GH│ │GHI│ │HIJ│ │IJK│ │JKL│ │KL│ │
│ │MN│ │MNO│ │NOP│ │OPQ│ │PQR│ │QR│ │
│ └──┘ └───┘ └───┘ └───┘ └───┘ └──┘ │
│ ┌→─┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→─┐ │
│ ↓GH│ ↓GHI│ ↓HIJ│ ↓IJK│ ↓JKL│ ↓KL│ │
│ │MN│ │MNO│ │NOP│ │OPQ│ │PQR│ │QR│ │
│ │ST│ │STU│ │TUV│ │UVW│ │VWX│ │WX│ │
│ └──┘ └───┘ └───┘ └───┘ └───┘ └──┘ │
│ ┌→─┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→─┐ │
│ ↓MN│ ↓MNO│ ↓NOP│ ↓OPQ│ ↓PQR│ ↓QR│ │
│ │ST│ │STU│ │TUV│ │UVW│ │VWX│ │WX│ │
│ └──┘ └───┘ └───┘ └───┘ └───┘ └──┘ │
└∊──────────────────────────────────┘

## Key

In [123]:
{⍺,⍵}⌸'Mississippi'  ⍝ Show indexes of items, by item

┌→──────────┐
↓M 1 │
│i 2 5 8 11│
│s 3 4 6 7│
│p 9 10 │
└+──────────┘

In [124]:
{⍺,≢⍵}⌸'Mississippi' ⍝ Frequencies

┌→──┐
↓M 1│
│i 4│
│s 4│
│p 2│
└+──┘

...or as tacit

In [125]:
(,∘≢⌸)'Mississippi'

┌→──┐
↓M 1│
│i 4│
│s 4│
│p 2│
└+──┘

Pick the most frequent item, returning all if several

In [126]:
{(⊣/m)/⍨frq⍷⍨frq⌷⍨⊃⍒frq←⊢/m←(,∘≢⌸)⍵} 'Mississippi' ⍝ Most frequent

┌→─┐
│is│
└──┘

## Sliding Tiles example

With ample help from @ngn on APL Orchard.

In [127]:
]dinput
moves←{
     d←(0 1)(1 0)(0 ¯1)(¯1 0)      ⍝ Move offsets
     m←⍵
     {⌽@i ⍵⊢m}¨(,⍳⍴m)∩d+⊂i←⊃⍸0=m
 }

In [130]:
state←¯1+4 4⍴?⍨16
moves state

┌→──────────────────────────────────────────┐
│ ┌→──────────┐ ┌→──────────┐ ┌→──────────┐ │
│ ↓ 3 0 2 8│ ↓ 3 2 8 0│ ↓ 3 2 5 8│ │
│ │11 15 5 7│ │11 15 5 7│ │11 15 0 7│ │
│ │13 10 9 12│ │13 10 9 12│ │13 10 9 12│ │
│ │ 1 4 14 6│ │ 1 4 14 6│ │ 1 4 14 6│ │
│ └~──────────┘ └~──────────┘ └~──────────┘ │
└∊──────────────────────────────────────────┘

A lot to decode here. The `i` expression gives the row-col of the 0 element:

In [131]:
state ⋄ ⊃⍸0=state

┌→──────────┐
↓ 3 2 0 8│
│11 15 5 7│
│13 10 9 12│
│ 1 4 14 6│
└~──────────┘
┌→──┐
│1 3│
└~──┘

Adding the offsets gives all potential moves:

In [132]:
d←(0 1)(1 0)(0 ¯1)(¯1 0)
d+⊂i←⊃⍸0=state                ⍝ disclose-enclose as we want the enclosed version later

┌→────────────────────────┐
│ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ │
│ │1 4│ │2 3│ │1 2│ │0 3│ │
│ └~──┘ └~──┘ └~──┘ └~──┘ │
└∊────────────────────────┘

Given the shape of m we can generate all valid coordinate pairs

In [133]:
,⍳⍴state

┌→────────────────────────────────────────────────────────────────────────────────────────────────┐
│ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ │
│ │1 1│ │1 2│ │1 3│ │1 4│ │2 1│ │2 2│ │2 3│ │2 4│ │3 1│ │3 2│ │3 3│ │3 4│ │4 1│ │4 2│ │4 3│ │4 4│ │
│ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ │
└∊────────────────────────────────────────────────────────────────────────────────────────────────┘

and then discard the moves that falls out of range as an intersection:

In [134]:
(,⍳⍴state)∩d+⊂i←⊃⍸0=state

┌→──────────────────┐
│ ┌→──┐ ┌→──┐ ┌→──┐ │
│ │1 2│ │1 4│ │2 3│ │
│ └~──┘ └~──┘ └~──┘ │
└∊──────────────────┘

We now map `{⌽@i ⍵⊢m}` over the list of valid moves. How does that work?

In [135]:
{⌽@i ⍵⊢state}¨(,⍳⍴state)∩d+⊂i←⊃⍸0=state

┌→──────────────────────────────────────────┐
│ ┌→──────────┐ ┌→──────────┐ ┌→──────────┐ │
│ ↓ 3 0 2 8│ ↓ 3 2 8 0│ ↓ 3 2 5 8│ │
│ │11 15 5 7│ │11 15 5 7│ │11 15 0 7│ │
│ │13 10 9 12│ │13 10 9 12│ │13 10 9 12│ │
│ │ 1 4 14 6│ │ 1 4 14 6│ │ 1 4 14 6│ │
│ └~──────────┘ └~──────────┘ └~──────────┘ │
└∊──────────────────────────────────────────┘

The dyadic `@` glyph in this case applies the reverse function `⌽` on the set of indexes given to its right argument. For example, if we want to switch the elements at (1 1) and (1 2) in m, we could say:

In [137]:
state ⋄ (⌽@(1 1) (1 2)) state

┌→──────────┐
↓ 3 2 0 8│
│11 15 5 7│
│13 10 9 12│
│ 1 4 14 6│
└~──────────┘
┌→──────────┐
↓ 2 3 0 8│
│11 15 5 7│
│13 10 9 12│
│ 1 4 14 6│
└~──────────┘

Using the right-tack `⊢` we can drop the brackets.

In [138]:
⌽@(1 1) (1 2) ⊢ state

┌→──────────┐
↓ 2 3 0 8│
│11 15 5 7│
│13 10 9 12│
│ 1 4 14 6│
└~──────────┘

## A heap implemented as a Leftist Tree

A heap is a data structure that can be implemented as a Leftist Tree, which lends itself naturally to a recursive implementation.

https://en.wikipedia.org/wiki/Leftist_tree
http://typeocaml.com/2015/03/12/heap-leftist-tree/

Here is a partial port of the OCaml version.

In [9]:
]dinput
Pop←{ ⍝ Pop off smallest element from a leftist tree ⍵
    0=≢⍵:⍬
    (value left right)←1↓⍵
    (left Merge right) value
}

In [10]:
]dinput
Push←{ ⍝ Insert item ⍵ into leftist tree ⍺, returning the resulting tree
    ⍺←⍬              ⍝ default to init
    1 ⍵ ⍬ ⍬ Merge ⍺ 
}

In [11]:
]dinput
Merge←{ ⍝ Merge leftist trees ⍺ and ⍵
    0=≢⍺:⍵ ⋄ 0=≢⍵:⍺                                 ⍝ If either is a leaf, return the other
    (key left right)←1↓⍺
    key>(⎕IO+1)⊃⍵:⍵∇⍺                               ⍝ Flip to ensure smallest is root of merged
    merged←right∇⍵                                  ⍝ Merge rightwards
    leftRank←⊃left ⋄ mergedRank←⊃merged
    leftRank≥⊃merged:(1+mergedRank) key left merged ⍝ Right is shorter
    (1+leftRank) key merged left                    ⍝ Left is shorter; make it the new right
}

In [12]:
⍝ Example given in http://typeocaml.com/2015/03/12/heap-leftist-tree/
h←Push 2
h Push←10
h Push←9

s←Push 3
s Push←6

h Merge s

┌─┬─┬─────────────────────┬────────┐
│2│2│┌─┬─┬───────┬───────┐│┌─┬──┬┬┐│
│ │ ││2│3│┌─┬─┬┬┐│┌─┬─┬┬┐│││1│10││││
│ │ ││ │ ││1│6│││││1│9│││││└─┴──┴┴┘│
│ │ ││ │ │└─┴─┴┴┘│└─┴─┴┴┘││ │
│ │ │└─┴─┴───────┴───────┘│ │
└─┴─┴─────────────────────┴────────┘

More compact, and a more natural fit for APL. Note that the Leftist Tree has slightly different performance characteristics to the standard heap. 

## Strings

Lower/upper-case a string

In [34]:
(819⌶) 'Hello All People in The World'   ⍝ NOTE: deprecated in Dyalog 18.X in favour of ⎕C

┌→────────────────────────────┐
│hello all people in the world│
└─────────────────────────────┘

In [35]:
1(819⌶) 'Hello All People in The World'  ⍝ NOTE: deprecated in Dyalog 18.X in favour of ⎕C

┌→────────────────────────────┐
│HELLO ALL PEOPLE IN THE WORLD│
└─────────────────────────────┘

We can also use regular expressions, of course:

In [36]:
{⊃('.+' ⎕S '\u0')⍵} 'Hello All People in The World'

┌→────────────────────────────┐
│HELLO ALL PEOPLE IN THE WORLD│
└─────────────────────────────┘

As we saw earlier, we can split a string on a regex pattern:

In [37]:
RegSplit←{(⊢/¨r)↓¨⍵⊂⍨(⍳≢⍵)∊1+⊃¨r←(⍺,'|^')⎕S 0 1⊢⍵}

In [38]:
'/+' RegSplit 'here/////be/dragons/'

┌→──────────────────────────┐
│ ┌→───┐ ┌→─┐ ┌→──────┐ ┌⊖┐ │
│ │here│ │be│ │dragons│ │ │ │
│ └────┘ └──┘ └───────┘ └─┘ │
└∊──────────────────────────┘

...or simply on a separator char

In [39]:
' ' (≠⊆⊢) 'hello world out there'

┌→──────────────────────────────┐
│ ┌→────┐ ┌→────┐ ┌→──┐ ┌→────┐ │
│ │hello│ │world│ │out│ │there│ │
│ └─────┘ └─────┘ └───┘ └─────┘ │
└∊──────────────────────────────┘

### Binary operations

Convert a decimal number to binary:

In [15]:
(2∘⊥⍣¯1) 54
(16⍴2)⊤54      ⍝ Fixed number of bits

┌→──────────┐
│1 1 0 1 1 0│
└~──────────┘

┌→──────────────────────────────┐
│0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 0│
└~──────────────────────────────┘

Binary to decimal:

In [17]:
2⊥0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 0

54

The boolean functions ∨∧⍲⍱≠~ etc all work as expected on binary vectors

In [9]:
0 0 0 0 1 0 ∧ 1 1 0 1 1 0 

┌→──────────┐
│0 0 0 0 1 0│
└~──────────┘

To right, and left shift we can use the following:

In [10]:
Lshift←{(≢⍺)↑⍵↓⍺}
Rshift←{(-≢⍺)↑(-⍵)↓⍺}

In [12]:
1 1 0 1 1 0 Rshift 2
1 1 0 1 1 0 Lshift 2

┌→──────────┐
│0 0 1 1 0 1│
└~──────────┘

┌→──────────┐
│0 1 1 0 0 0│
└~──────────┘

### TODO UNSORTED

83 ¯1⎕MAP filename ⍝ read raw bytes
{⍺⎕UCS¨⍨⊂'-\w+$'⎕R''⊢⍵}/2↑⎕NGET 'data/2015/08.txt' 1 ⍝ read raw bytes

(∪⍳⊢)Y ⍝ convert strings to IDs
a⌹a=a  average
(⍕≡⊢)Y ⍝ is Y simple character array?
(1=2|⎕DR)Y ⍝ is Y all numeric

v←{A←⊃∘.+/3⍴⊂⍳100 ⋄ ⊃,/{⍵,¨⍸(100-⍵)=A}¨⍳100}⍬ ⍝ stars & bars

best⊣⍣(~value<best)←value  ⍝ if value < best then best=value

### JSON

Parse a JSON-string into nested namespaces where needed:

In [32]:
JSONSTR←'{"key11":"value11","key12":{"key121":"value121"},"key13":[1,2,3]}'
⎕←data←⎕JSON JSONSTR

#.[JSON object]

This contains the names of simple keys and the names of references (dicts) as separate entries in the namespace:

In [35]:
data.(⎕NL ¯2)  ⍝ Simple keys
data.(⎕NL ¯9)  ⍝ Object references

┌→────────────────┐
│ ┌→────┐ ┌→────┐ │
│ │key11│ │key13│ │
│ └─────┘ └─────┘ │
└∊────────────────┘

┌→────────┐
│ ┌→────┐ │
│ │key12│ │
│ └─────┘ │
└∊────────┘

Negating the argument to ⎕NL simply means return as a nested vector, rather than as an array.

To get the corresponding values, we simply execute the entries returned by ⎕NL:

In [38]:
⎕←simpleValues←data.(⍎¨⎕NL ¯2)
⎕←objectValues←data.(⍎¨⎕NL ¯9)

┌→──────────────────┐
│ ┌→──────┐ ┌→────┐ │
│ │value11│ │1 2 3│ │
│ └───────┘ └~────┘ │
└∊──────────────────┘

┌→──────────────────────────────┐
│ #.[JSON object].[JSON object] │
└#──────────────────────────────┘

In [5]:
⍝ CHECK: (↑⍣≡0∘⎕JSON)Y
⍝ (1⎕JSON{1<≢⍴⍵:∇¨⊂⍤¯1⊢⍵ ⋄ ⍵})Y apl->json

 Anonymize json by replacing all values by the same type, but random

In [5]:
rndjson←{'[0-9]'⎕R{⎕D[1+?9]}'"([^"]|\.)*".'⎕R{':'=t←⊃⌽m←⍵.Match:m⋄t,⍨'"','"',⍨⎕a[?26⍴⍨¯3+≢m]}⍵}

### Namespaces

Namespaces is the way to create objects, records or structs in Dyalog.

In [41]:
a←⎕NS''  ⍝ Create an anonymous namespace                 

In [43]:
a.(Name City)← 'Stefan' 'Bristol'  ⍝ Assign two fields
a.Name ⋄ a.City

┌→─────┐
│Stefan│
└──────┘
┌→──────┐
│Bristol│
└───────┘

### Introspection and typing

In [44]:
IsNumber←{(1=2|⎕DR)⍵}                     ⍝ Are we a number?
IsString←⍕≡⊢                              ⍝ Are we a string?
IsNamespace←{(326=⎕DR⍵)}                  ⍝ Are we a reference/object/namespace?

### Performance comparisons

In [16]:
b←↑{(⍵ 1) (0 0)}¨⍳10000
vs←⍳10000
rs←?⍨10000

]runtime -c "vs,¨⍨@1⍤0 1@rs⊢b" "(,¨∘vs)@(rs,¨1)⊢b"

┌→────────────────────────────────────────────────────────────────────────────┐
↓ │
│ vs,¨⍨@1⍤0 1@rs⊢b → 2.7E¯2 | 0% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕ │
│ (,¨∘vs)@(rs,¨1)⊢b → 4.8E¯3 | -82% ⎕⎕⎕⎕⎕⎕⎕ │
└─────────────────────────────────────────────────────────────────────────────┘

### Grouping

Given a vector of integer pairs:

    m1←(5 3)(5 6)(7 5)(4 7)(1 8)(2 4)(1 2)

Group based on the first element of each pair to produce:

    m2←(5 (3 6))(7 (,5)) (4 (,7)) (1 (8 2)) (2 (,4))
    
Method 1: sort, and partition-enclose:

In [2]:
m1←(5 3)(5 6)(7 5)(4 7)(1 8)(2 4)(1 2)
data←(↑m1)[⍋(↑m1)[;1];]
k←⊣/data
↓⍉↑(∪k)(k⊆⊢/data)

┌───────┬─────┬─────┬───────┬─────┐
│┌─┬───┐│┌─┬─┐│┌─┬─┐│┌─┬───┐│┌─┬─┐│
││1│8 2│││2│4│││4│7│││5│3 6│││7│5││
│└─┴───┘│└─┴─┘│└─┴─┘│└─┴───┘│└─┴─┘│
└───────┴─────┴─────┴───────┴─────┘

Method 2: use key:

In [3]:
(⊃¨{⊂⍺⍵}⌸⊢/¨)m1

┌───────┬─────┬─────┬───────┬─────┐
│┌─┬───┐│┌─┬─┐│┌─┬─┐│┌─┬───┐│┌─┬─┐│
││5│3 6│││7│5│││4│7│││1│8 2│││2│4││
│└─┴───┘│└─┴─┘│└─┴─┘│└─┴───┘│└─┴─┘│
└───────┴─────┴─────┴───────┴─────┘

or more tacitly,

In [4]:
(↓⊃¨,∘⊂⌸⊢/¨)m1

┌───────┬─────┬─────┬───────┬─────┐
│┌─┬───┐│┌─┬─┐│┌─┬─┐│┌─┬───┐│┌─┬─┐│
││5│3 6│││7│5│││4│7│││1│8 2│││2│4││
│└─┴───┘│└─┴─┘│└─┴─┘│└─┴───┘│└─┴─┘│
└───────┴─────┴─────┴───────┴─────┘

### Odd one out

Given a simple vector where all items are equal, bar one, find the index of the odd one out.

In [14]:
vect←243 243 243 243 251 243 243

Naively, use key `⌸` to make the histogram, find the location of the single 1 in the counts column, pick the corresponding value and then locate the index in the vector:

In [16]:
hist←{⍺,≢⍵}⌸d
oneIndex←hist[;2]⍳1
oddVal←hist[oneIndex;1]
vect⍳oddVal

5

We can compress this basic approach by doing more work in the key operand:

In [17]:
∊{⊂⍵/⍨1=≢⍵}⌸vect

5

This finds the places where the count is 1 and emits the corresponding index. Recall that the `⍵` in the operand is a set of indexes for each unique element:

In [18]:
{⍺ ⍵}⌸vect

┌───┬───────────┐
│243│1 2 3 4 6 7│
├───┼───────────┤
│251│5 │
└───┴───────────┘

A different approach is to reduce the indices with intersect `∩`:

In [19]:
∊{∩/⍵}⌸vect

5

It's perhaps not obvious how this works. We know that the elements of `⍵` will be unique, so the reduction will always be empty, unless the vector contains a single element:

In [9]:
∊∩/1 2 3 4 6 7

In [10]:
∊∩/5

5

A completely different approach is to look for elements in the vector that differs from both its neighbours, wrapping around at the edges:

In [20]:
⍸{⍵∧1⌽⍵}∘{⍵≠¯1⌽⍵}vect

5

Another suggestion from the APLOrchard:

In [22]:
(1⍳⍨1⊥∘.=⍨)vect

5

In [24]:
assert←{⍺←'assertion failure' ⋄ 0∊⍵:⍺ ⎕signal 8 ⋄ shy←0}

Index of smallest element

In [3]:
elems⍳⌊/elems←6 2 8 9 3 1 10 15
⊃⍋elems

6

6

### Partition a matrix into non-overlapping sub-matrices -- and re-merge

In [18]:
⊢∘⊂⌺(2 2⍴2 2)⊢4 4⍴⍳16

┌───┬───┬─────┬─────┐
│0 1│2 3│ 8 9│10 11│
│4 5│6 7│12 13│14 15│
└───┴───┴─────┴─────┘

In [20]:
1 1↓⊢∘⊂⌺(2 2⍴3 3)⊢0⍪0⍪0,0,12 12⍴⍳144

┌───────────┬───────────┬───────────┬───────────┐
│ 1 2 3 │ 4 5 6 │ 7 8 9 │10 11 12 │
│13 14 15 │16 17 18 │19 20 21 │22 23 24 │
│25 26 27 │28 29 30 │31 32 33 │34 35 36 │
├───────────┼───────────┼───────────┼───────────┤
│37 38 39 │40 41 42 │43 44 45 │46 47 48 │
│49 50 51 │52 53 54 │55 56 57 │58 59 60 │
│61 62 63 │64 65 66 │67 68 69 │70 71 72 │
├───────────┼───────────┼───────────┼───────────┤
│73 74 75 │ 76 77 78│ 79 80 81│ 82 83 84│
│85 86 87 │ 88 89 90│ 91 92 93│ 94 95 96│
│97 98 99 │100 101 102│103 104 105│106 107 108│
├───────────┼───────────┼───────────┼───────────┤
│109 110 111│112 113 114│115 116 117│118 119 120│
│121 122 123│124 125 126│127 128 129│130 131 132│
│133 134 135│136 137 138│139 140 141│142 143 144│
└───────────┴───────────┴───────────┴───────────┘

In [11]:
⎕IO←1 ⋄ 2 2 {⊂[2×⍳≢⍴⍵]⍵⍴⍨∊⍺,¨⍨⍺÷⍨⍴⍵} 4 4 ⍴ ⍳16

┌─────┬─────┐
│1 2 │3 4 │
│5 6 │7 8 │
├─────┼─────┤
│ 9 10│11 12│
│13 14│15 16│
└─────┴─────┘

In [9]:
⊂⍤2⊢1 3 2 4⍉2 3 2 3⍴6 6⍴⍳36

┌────────┬────────┐
│ 1 2 3│ 4 5 6│
│ 7 8 9│10 11 12│
│13 14 15│16 17 18│
├────────┼────────┤
│19 20 21│22 23 24│
│25 26 27│28 29 30│
│31 32 33│34 35 36│
└────────┴────────┘

To merge sub-matrices back to the original state:

In [25]:
m←1 1↓⊢∘⊂⌺(2 2⍴3 3)⊢0⍪0⍪0,0,12 12⍴⍳144
Merge←{((⍴×⍴∘⊃)⍴1 3 2 4⍉↑)⍵}
Merge m

1 2 3 4 5 6 7 8 9 10 11 12
 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36
 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60
 61 62 63 64 65 66 67 68 69 70 71 72
 73 74 75 76 77 78 79 80 81 82 83 84
 85 86 87 88 89 90 91 92 93 94 95 96
 97 98 99 100 101 102 103 104 105 106 107 108
109 110 111 112 113 114 115 116 117 118 119 120
121 122 123 124 125 126 127 128 129 130 131 132
133 134 135 136 137 138 139 140 141 142 143 144

In [1]:
m←10⍴5↑1 ⋄ ↑m∘⊂¨m⊂[1]10 10⍴⍳100

┌──────────────┬───────────────┐
│ 1 2 3 4 5│ 6 7 8 9 10 │
│11 12 13 14 15│16 17 18 19 20 │
│21 22 23 24 25│26 27 28 29 30 │
│31 32 33 34 35│36 37 38 39 40 │
│41 42 43 44 45│46 47 48 49 50 │
├──────────────┼───────────────┤
│51 52 53 54 55│56 57 58 59 60│
│61 62 63 64 65│66 67 68 69 70│
│71 72 73 74 75│76 77 78 79 80│
│81 82 83 84 85│86 87 88 89 90│
│91 92 93 94 95│96 97 98 99 100│
└──────────────┴───────────────┘

### Neighbourhoods

In [21]:
v←⍸×3 3 ⍴ 5 0 4 0 0 3 0 2 0 ⍝ All non-zero coord pairs
n4←{⍺∩⍵(+,-)(0 1)(1 0)} ⍝ or possibly better: {(⍸⍺)∩⍵(+,-)(0 1)(1 0)} 
n8←{⍺∩⍵(+,-)(0 1)(1 0)(¯1 1)(1 1)}
Move←{('NESW'⍳⍵)⊃(¯1 0)(0 1)(1 0)(0 ¯1)}

In [23]:
v n4 ⊂2 3

┌───┐
│1 3│
└───┘

## 4-connected neighbours via shifting

In [10]:
shifts←{(⍺,2⊣/⍵)(2⊢/⍵,⍺)(2⊢⌿⍵⍪⍺)(⍺⍪2⊣⌿⍵)}  ⍝ 4-connected neighbours via shifting

In [3]:
⊢m←5 5⍴25?25
⊢s←¯1 shifts m

┌→─────────────┐
↓ 4 1 22 20 16│
│10 25 7 18 11│
│15 2 12 23 9│
│17 6 14 21 19│
│ 3 8 5 13 24│
└~─────────────┘

┌→────────────────────────────────────────────────────────────────────┐
│ ┌→─────────────┐ ┌→─────────────┐ ┌→─────────────┐ ┌→─────────────┐ │
│ ↓¯1 4 1 22 20│ ↓ 1 22 20 16 ¯1│ ↓10 25 7 18 11│ ↓¯1 ¯1 ¯1 ¯1 ¯1│ │
│ │¯1 10 25 7 18│ │25 7 18 11 ¯1│ │15 2 12 23 9│ │ 4 1 22 20 16│ │
│ │¯1 15 2 12 23│ │ 2 12 23 9 ¯1│ │17 6 14 21 19│ │10 25 7 18 11│ │
│ │¯1 17 6 14 21│ │ 6 14 21 19 ¯1│ │ 3 8 5 13 24│ │15 2 12 23 9│ │
│ │¯1 3 8 5 13│ │ 8 5 13 24 ¯1│ │¯1 ¯1 ¯1 ¯1 ¯1│ │17 6 14 21 19│ │
│ └~─────────────┘ └~─────────────┘ └~─────────────┘ └~─────────────┘ │
└∊────────────────────────────────────────────────────────────────────┘

Now we can say that the 4-connected neighbours of m[1;1] are:

In [4]:
(↑s)[;1;1]

┌→─────────┐
│¯1 1 10 ¯1│
└~─────────┘

## Grid to flat vector of neighbours

In [14]:
nvec←{{⍵~¯1}¨,↓2 0 1⍉↑¯1shifts(⍴⍵)⍴(⍳×/⍴⍵)} ⍝ using shifts function above

In [15]:
nvec 3 3⍴1

┌───┬─────┬───┬─────┬───────┬─────┬───┬─────┬───┐
│1 3│0 2 4│1 5│4 6 0│3 5 7 1│4 8 2│7 3│6 8 4│7 5│
└───┴─────┴───┴─────┴───────┴─────┴───┴─────┴───┘

## Alpush/Alset

In [1]:
alupsert←{(k v)←⍵⋄3::⍺ alpush k v⋄⍺ alset k v} ⍝ Modify if present
algetd←{3::¯1↑⍵⋄⍺ alget (⊃⍵)}                  ⍝ Get if present, else return given default

done

Rebuilding user command cache... done

Atop: One way to look at f∘g vs f⍤g is that when given a left argument, ∘ gives it to the left-hand function and ⍤ gives it to the right-hand function. Other than that, they are equivalent

Another way to look at f∘g vs f⍤g is simply choosing order of the first two tokens: X f∘g Y computes X f g Y and X f⍤g Y computes f X g Y
 
Over: So, remember how f∘g preprocesses the right argument of f using g?
One way to look at Over is simply as preprocessing all arguments of f using g.
"All" as in _both or the only_.

So again f⍥g Y is the same as f⍤g Y and f∘g Y. The difference is again when we do a dyadic application.

So while X f∘g Y is X f(g Y) we have X f⍥g Y be (g X)f(g Y).

One of my favourites lately is ,⍥⊂ which I prefer over {⍺ ⍵}

In [4]:
'YYYY DD MM hhmm'(1200⌶)1⎕DT⊂2020 08 11 11 32 ⍝ Dyalog 18 date time

┌───────────────┐
│2020 11 08 1132│
└───────────────┘

### Constrain a set of coordinates to a rectangle

In [6]:
⎕io←0
YMIN←0 ⋄ YMAX←50 ⋄ XMIN←0 ⋄ XMAX←50
c←(23 43)(78 34)(¯45 34)

In [7]:
⊃∧/0=(YMIN YMAX) (XMIN XMAX)⍸¨↓⍉↑c

1 0 0

In [8]:
(⊢≡YMIN XMIN⌈YMAX XMAX⌊⊢)¨c          ⍝ Slower

1 0 0

## Noddy hash table

In [12]:
Hash←{keys←(1500⌶),⍺⋄vals←,⍵⋄{⍵⊣⍵.(Vals Keys Default)←vals keys ⍬}⎕NS''}
In←{⍺∊⍵.Keys}

In [13]:
]dinput
Get←{
    ~⍵ In ⍺:⍺.Default
    ⍺.Vals[⍺.Keys⍳⍵]
}

In [14]:
]dinput
Set←{ ⍝ Upsert ht ⍺ with kv pair ⍵. ⍝ Returns 1 if this is a new key; 0 otherwise.
    ht←⍺
    (k v)←⍵
    i←ht.Keys⍳⊂k
    i=≢ht.Keys:1⊣ht.Keys,←⊂k⊣ht.Vals,←v ⍝ New key
    ht.Vals[i]←v                        ⍝ Replace existing
    0
}

In [15]:
]dinput
Append←{ ⍝ Append a value to those belonging to the key. If new key, insert kv pair
    ht←⍺
    (k v)←⍵
    i←ht.Keys⍳⊂k
    i=≢ht.Keys:1⊣ht.Keys,←⊂k⊣ht.Vals,←v ⍝ New key
    ht.Vals[i],←v                       ⍝ Append to existing
    0
}

### Space ship compares (<=>)

In [8]:
ssc1←{⊃¯1 0 1[⍸⍺(<,=,>)⍵]}
ssc2←>-<
ssc3←×-

### Tuples

In [9]:
tuples←{↓(⍺÷⍨≢⍵) ⍺⍴⍵} ⍝ errors if not equally dividable
tuples←{⍵⊂⍨(≢⍵)⍴⍺↑1}
tuples←⊢⊂⍨≢⍤⊢⍴⊣↑≢

### Indices of unique

In [10]:
⍸≠ 1 1 2 1 3 4 5 2 1

1 3 5 6 7

### Namespace as dict

In [1]:
_Gets←{⍎'⍺⍺.',⍺,'←⍵'⊣⍺⍺}

done

In [3]:
ns←⎕ns''
'hello'(ns _Gets)'world'
ns.hello
ns⍎'hello'

world

world

### Mapping between 2D and 1D coordinates

When you flatten (,⍵) a matrix m with shape ⍴m its set of coordinates changes from ⍳⍴m to ⍳×/⍴m. They become simply the indices of a vector. The mapping between the two can be done with k←(⍴m)⊥i j and its inverse i j←(⍴m)⊤k, where i j are a pair of indices in m, and k is an index in ,m

In [3]:
⊢m←3 5 ⍴15?15

1 7 14 9 13
2 12 4 3 15
5 8 6 10 11

In [4]:
⊢ravel←,m

1 7 14 9 13 2 12 4 3 15 5 8 6 10 11

In [6]:
⊢k←(⍴m)⊥1 4 ⍝ 2D to 1D

9

In [7]:
k⊃ravel

3

In [8]:
⊢i j←(⍴m)⊤k ⍝ 1D to 2D

1 4

In [10]:
m[⊂i j]

9

### Generalised padding

Center a matrix of rank n in a larger matrix of rank n+1.

In [18]:
⊢d←3 3⍴0 1 0 0 0 1 1 1 1

0 1 0
0 0 1
1 1 1

In [2]:
Pad←{(-⍵+⍵+⍴⍺)↑(⍵+⍴⍺)↑⍺}

In [7]:
d {(-(2×⍵)+⍴⍺)↑(⍵+⍴⍺)↑⍺} 5

0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 1 0 0 0 0 0 0
0 0 0 0 0 0 0 1 0 0 0 0 0
0 0 0 0 0 1 1 1 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0

Here's a different approach:

In [18]:
Pad2←{⍵@(1+⍳⍴⍵)⊢0⍴⍨2+⍴⍵} ⍝ https://aplcart.info/?q=surround%20array#

In [19]:
Pad2 1 2 3

0 1 2 3 0

In [20]:
Pad2 d

0 0 0 0 0
0 0 1 0 0
0 0 0 1 0
0 1 1 1 0
0 0 0 0 0

In [21]:
Pad2⍣5⊢d

0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 1 0 0 0 0 0 0
0 0 0 0 0 0 0 1 0 0 0 0 0
0 0 0 0 0 1 1 1 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0

In [22]:
Pad2 (↑,∘⊂)d ⍝ Increase rank first

0 0 0 0 0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0
 
0 0 0 0 0
0 0 1 0 0
0 0 0 1 0
0 1 1 1 0
0 0 0 0 0
 
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0

Better:

In [19]:
Pad3←{0,0,⍨0⍪0⍪⍨⍵}
Pad3⍣5⊢d

0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 1 0 0 0 0 0 0
0 0 0 0 0 0 0 1 0 0 0 0 0
0 0 0 0 0 1 1 1 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0

In [21]:
Pad4←{(⍺+⍺+⍴⍵)↑(-⍺+⍴⍵)↑⍵} ⍝ ⍺ is the pad amount

### TODO
@voidhawk on AoC Day16

In [ ]:
⎕IO←1 ⍝ voidhawk
p←(⊢⊆⍨0≠≢¨)⊃⎕NGET'data/2020/day16.txt'1
ds←{⍎¨⍵(∊⊆⊣)⎕D}
rs←ds¨⊃p ⋄ yt ot←{↑ds¨1↓⍵⊃p}¨2 3
ir←{a b c d←⍵ ⋄ ((d≥⍺)∧c≤⍺)∨(b≥⍺)∧a≤⍺}
+/ot[⍸~e←∨/ot∘.ir rs] ⍝ part 1
m←∧⌿rs∘.ir⍨yt⍪ot⌿⍨∧/e
×/6↑(,yt)⌷⍨⊂⍋(⊂⍋⍋+/m)⌷∊~⍨\⍸¨↓m[⍋+/m;] ⍝ part 2

### Corners of 2D Array

In [3]:
⊢m←3 3⍴9?9
{↑(⊣/⍵)(⊢/⍵)}⍣2⊢m

4 3 1
9 7 5
8 2 6

4 1
8 6

In [4]:
⌷⍨∘⊃⍨⍤0 99 ⍝ Sane indexing

⍤ 
 ┌┴┐ 
 ⍨ 0 99
 ┌─┘ 
 ∘ 
 ┌┴┐ 
 ⍨ ⊃ 
┌─┘ 
⌷

In [5]:
(⊢⊂⍨1,2≠/⊢) ⍝ map of changes

┌─┼───┐ 
⊢ ⍨ ┌─┼───┐ 
┌─┘ 1 , ┌─┼─┐
⊂ 2 / ⊢
 ┌─┘ 
 ≠

In [6]:
(∨.∧⍨∨⊢)⍣≡ ⍝ Transitive closure on APL Cart?
⍝ if G is the adjacency matrix of a graph, ∨.∧⍨⍣≡G is its transitive closure; 
⍝ for a weighted graph, ⌊.⌈⍨⍣≡G is shortest paths

⍣ 
 ┌─┴─┐
 ┌─┼─┐ ≡
 ⍨ ∨ ⊢ 
 ┌─┘ 
 . 
┌┴┐ 
∨ ∧

In [7]:
3{0 ⍺⊤⍵}19 ⍝ divmod

6 1

Rotate a matrix by 1 step, rather than by 90 degrees:

In [1]:
]dinput
mat←Rot mat;size;ranges;order;n;mask;rings;cutoffs;flat;grade
size←≢mat
rings←,(⌽⌊⊖)⌊/¨⍳2⍴size
order←,{⌽⍉,⌸⍵+≢⍵}⍣2⍣size⍪⍬
:For n :In ∪,rings
   mask←n=,rings
   grade←⍋mask/order
   (mask/,mat)←(1⌽(mask/,mat)[grade])[⍋grade]
:EndFor

In [ ]:
⍝ Crypto library
]cd /Users/stefan/work/dyalog/DCLmacP
)load dyacrypt.dws
./dyacrypt.dws saved Tue Mar  6 14:19:40 2018
      RUN
      
#.Crypt.Init
#.Win.HexTxt #.Crypt.HASH_MD5 #.Crypt.Hash 'Hello, world'

In [2]:
⍝ Faster? pmat from dzaima
dpmat←{1=⍵:,⊂,0 ⋄ (⊃,/)¨(⍳⍵)⌽¨⊂(⊂(!⍵-1)⍴⍵-1),⍨∇⍵-1} ⍝ random order - v fast
sdpmat←{{(⊂⍋⍵)⌷⍵}⍉↑{1=⍵:,⊂,0 ⋄ (⊃,/)¨(⍳⍵)⌽¨⊂(⊂(!⍵-1)⍴⍵-1),⍨∇⍵-1}⍵} ⍝ same order as dfns.pmat

In [8]:
m←3 3⍴9?9
⎕SE.Dyalog.Utils.repObj m

3 3⍴4 1 6 5 2 9 7 8 3

## Solve maze

In [13]:
'span' 'stpath'⎕CY'dfns'
⎕io←0

In [14]:
:Namespace graph
    ⎕io←0
    ⍝ https://www.dcode.fr/maze-generator
    N←{(⍸⍺)∩(⊂⍵)(+,-)(0 1)(1 0)}

    _solve←{
        (n g)←{f({{f⍳⊂⍵}¨⍵}¨m∘N¨f←⍸m←~'#'=⍵)}⍺⍺
        n[(g ##.span ⍺) ##.stpath ⍵]
    }

    showpath←{
        '⎕'@⍺⊢⍵
    }

:EndNamespace

In [15]:
maze←↑⊃⎕NGET'/Users/stefan/work/dyalog/learnapl/maze-big.txt'1

In [17]:
p ← 0 (maze graph._solve) 5004

In [18]:
p graph.showpath maze

⎕.###################################################################################################
⎕⎕#⎕⎕⎕⎕⎕⎕⎕⎕⎕#..⎕⎕⎕#..⎕⎕⎕#.....#...........#...#.....#.......#.#...#.....#.....#.#.#.....#.#.#.......#
#⎕#⎕###.###⎕###⎕#⎕###⎕#⎕#####.###.#.###.###.#.#.#.###.###.###.###.#.###.#.#####.#.#.###.#.#.#.#######
#⎕⎕⎕#.....#⎕#..⎕#⎕⎕⎕⎕⎕#⎕⎕⎕#⎕⎕⎕⎕⎕⎕⎕#...#.#.#.#...#...#.#.....#...#.....#.#.#.....#.#.#.........#.....#
#####.#.###⎕###⎕#.#######⎕#⎕#####⎕#######.###.#.#####.#.#.#.#.#######.###.#.#.#.#.###.#########.###.#
#.#.#.#.#..⎕⎕⎕⎕⎕#.#...#.#⎕#⎕#.#.#⎕........#...#.....#.#.#.#.....#.#...#...#.#.#.................#.#.#
#.#.#####.#.#########.#.#⎕#⎕#.#.#⎕#########.#####.#.#.#.#########.#.#####.#.#####.#.#####.#####.#.###
#...#.....#.....#.......#⎕⎕⎕#...#⎕#⎕⎕⎕#.#.#.#.#...#.#.#.#...#.......#.#.....#.#.#.#.....#...#.......#
#.#############.#.#.#########.###⎕#⎕#⎕#.#.###.###.#.#.###.#########.#.#######.#.#.#.#####.#.###.#.#.#
#.#...#.....#.#...#.#.#.......#..⎕⎕⎕#⎕......#.#...#.#.................#...#.#.....#.....#.#.#.#.#.#.#
#.###.#.###.#.#######.#.###.#####.###⎕###.###.#.#.#.###.###.###.#.#######.#.#.#.#.#.###.###.#.#.#.#.#
#.....#...#.#...#.#.......#...#...#..⎕..#...#.#.#.#.#.#.#.....#.#.#.#...#.....#.#.#.#.#...#...#.#.#.#
###.#.#.#######.#.###.#####.#.###.###⎕#######.###.#.#.#.#######.###.#.#######.###.###.#.#######.#####
#...#...#.........#...#...#.#.....#.#⎕⎕⎕⎕⎕#.#.#...#.#.#.#.....#.......#.....#.#.....#.#.#...........#
#.###.#.#.#.#####.#######.#.#######.#.###⎕#.#.#.#.###.#.#.#############.#.###.#######.#.###.#.#.#.#.#
#.#...#.#.#.#...#.#...#...#.#...........#⎕#.#...#.......#...........#.#.#.........#.....#.#.#.#.#.#.#
#######.###.#.#######.#.#.#.#######.###.#⎕#.###########.#.#####.#####.###.#######.#####.#.###.#######
#.#.....#.#...#.#.......#.#.#.#.#.#...#.#⎕⎕⎕⎕⎕#.....#...#...#.#.....#.#.......#.#...#.....#.....#...#
#.#.#####.#.###.###.#######.#.#.#.#.#######.#⎕#.#.###.###.###.###.###.###.#####.###########.#######.#
#.....#.#.#.#...............#...#.#...#...#.#⎕#.#...#...#...#...............#.#.....#.....#.#...#...#
###.#.#.#.#.#.###.#####.###.#.#.#.#####.#####⎕###.#####.#########.###.#####.#.###.#.###.#######.#.###
#...#.#...#...#.#...#.....#...#...#.#.....#..⎕#.#...........#...#.#...#...........#...#.....#...#.#.#
#.#####.#####.#.#.#.#######.###.###.#####.###⎕#.#.###.#######.#.###.#######.#.#.#.#.###.###.#.###.#.#
#.....#.......#.#.#...#...#.#...#.#.#...#.#.#⎕⎕⎕....#.#...#...#.#...#.......#.#.#.#...#...#.#.......#
#.###.#####.#.#.###.#####.#.###.#.#.###.#.#.###⎕###.###.#####.#.###########.#########.#.#####.#####.#
#.#.......#.#.#.........#...#.#.#.......#....⎕⎕⎕#.....#.....#.#...#...........#.#.#.....#.......#.#.#
###.#.#####.#.#.###.#.###.###.#####.###.#####⎕#############.###.###.#.#.#.#####.#.###.#.###.#####.#.#
#...#.......#.#...#.#.#.........#.....#......⎕⎕⎕#.......#⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕#.#.#.#...#.#.....#.......#.....#
#.#####.###########.#.#########.#############.#⎕#.###.###⎕###.#.###⎕###.#.#.#.#.###.#####.#######.###
#.#.#.#.#...#.#.....#.#.......#.#...........#.#⎕#...#.#⎕⎕⎕#...#.#.#⎕..#.#...#...#...#.......#...#.#.#
#.#.#.###.#.#.#####.###.###.###.###.#####.#.###⎕#.#####⎕#######.#.#⎕#.#.#########.#.###.###.#.#.#.#.#
#.......#.#.........#...#...#.....#.#.....#.#..⎕⎕⎕⎕⎕⎕⎕⎕⎕..#...#.#.#⎕#.#.#.....#.#.#...#.#...#.#.#...#
###.#.###.###.###.###.###.#####.#.#####.#########.#####.###.#####.#⎕#######.###.#####.#########.#.###
#.#.#.#...#.....#.....#.....#...#...#.....#.#.#.....#...#...#⎕⎕⎕⎕⎕⎕⎕#.#.#.#...#.#...#.#.#.#.#.....#.#
#.###.#.#######.#.#.#.#.#####.#.###.#.#####.#.###.#.###.#.###⎕#######.#.#.#.###.###.#.#.#.#.#.###.#.#
#...#.#.#...#...#.#.#.#...#...#.#.#.#.....#.#.#.#.#.#...#..⎕⎕⎕#...#.#.....#.....#.#...#.....#.#...#.#
###.###.#.#.###.#######.###.#.#.#.#.#.###.#.#.#.###.#######⎕#.#.###.#.#.#####.#.#.#####.#.#.#######.#
#.........#.#.....#.#.....#.#.#...#.#.#.....#.#.#...#...#⎕⎕⎕#.....#.#.#.#...#.#.#.......#.#.#...#...#
#.#.#####.###.#####.#.###.###########.###.#.#.#.#####.#.#⎕#.#######.###.#.#.#.#####.#.#.###.###.#.#.#
#.#.#.#...#.#.....#...

## Bar charts from https://www.jsoftware.com/papers/APL_exercises/

In [20]:
{↑(⌈/⍵){(⍵/'⎕'),(⍺-⍵)/'.'}¨⍵} 3 0 9 6 2 7 6 ⍝ Bar chart

⎕⎕⎕......
.........
⎕⎕⎕⎕⎕⎕⎕⎕⎕
⎕⎕⎕⎕⎕⎕...
⎕⎕.......
⎕⎕⎕⎕⎕⎕⎕..
⎕⎕⎕⎕⎕⎕...

In [22]:
{m←⌈/⍵⋄⍵{⍵/⍨⍺,m-⍺}⍤0 1⊢'⎕.'} 3 0 9 6 2 7 6

⎕⎕⎕......
.........
⎕⎕⎕⎕⎕⎕⎕⎕⎕
⎕⎕⎕⎕⎕⎕...
⎕⎕.......
⎕⎕⎕⎕⎕⎕⎕..
⎕⎕⎕⎕⎕⎕...

In [31]:
(↑⍴¨∘'⎕')3 0 9 6 2 7 6

⎕⎕⎕ 
 
⎕⎕⎕⎕⎕⎕⎕⎕⎕
⎕⎕⎕⎕⎕⎕ 
⎕⎕ 
⎕⎕⎕⎕⎕⎕⎕ 
⎕⎕⎕⎕⎕⎕

which is the same as

In [32]:
↑{⍵⍴'⎕'}¨3 0 9 6 2 7 6

⎕⎕⎕ 
 
⎕⎕⎕⎕⎕⎕⎕⎕⎕
⎕⎕⎕⎕⎕⎕ 
⎕⎕ 
⎕⎕⎕⎕⎕⎕⎕ 
⎕⎕⎕⎕⎕⎕

In [1]:
]dinput
:Class Dict
    ⍝ )ed ○ Dict
    :Field Public _keys←(1500⌶)⍬
    :Field Public _values←⍬

    :Property Keyed Default Item
        :Access Public Instance

        ∇ r←get arg
            r←_values[_keys⍳⊃arg.Indexers] ⍝ Read components
        ∇

        ∇ set arg;dedupe;unique;kk;vv;seen
            ⍝ Find the unique indexes and corresponding vals
            unique ← ≠⊃arg.Indexers
            kk ← unique/⊃arg.Indexers
            vv ← unique/arg.NewValue
    
            ⍝ Separate out keys we already hold from new ones
            seen ← kk∊_keys
            (seen/_values)←seen/vv

            ⍝ Append new keys and corresponding values
            _keys,←(~seen)/kk
            _values,←(~seen)/vv
        ∇

    :EndProperty

    ∇ Make(keys values)
        :Access Public
        :Implements Constructor

        (_keys _values)←keys values
    ∇
:EndClass

In [4]:
z←⎕NEW Dict (('APL' 'Rocks!')(42 'Bob'))
z['Rocks!' 'APL' 'APL'] 

┌───┬──┬──┐
│Bob│42│42│
└───┴──┴──┘

## Using .NET

From https://docs.microsoft.com/en-us/dotnet/api/system.collections.hashtable?view=net-5.0

This should work, but doesn't seem to on a mac afaik.

In [ ]:
⎕USING←'System.Collections'
openWith←⎕NEW Hashtable
⍝ Add some elements to the hash table. There are no
⍝ duplicate keys, but some of the values are duplicates.
openWith.Add'txt' 'notepad.exe'
openWith.Add'bmp' 'paint.exe'
openWith.Add'dib' 'paint.exe'
openWith.Add'rtf' 'wordpad.exe'
openWith
(⌷openWith).(Key Value)
(openWith).(Keys Values)
System.Collections.Hashtable+KeyCollection  System.Collections.Hashtable+ValueCollection 
⌷¨(openWith).(Keys Values)

## Testing notes

* https://github.com/Dyalog/link/blob/master/StartupSession/Link/Test.dyalog
* https://github.com/abrudz/Get/blob/master/abrudz.dyalog#L248
* https://dfns.dyalog.com/n_test.htm
* Try e.g. ]uload calendar then )ed JSUtils.Test

## Read .clou file into namespace

In [3]:
∇ r←pc file;clou;_set;sect;sectns
clou←⊃⎕NGET file 1
_set←{⍎'⍺⍺.',⍺,'←⍵'⊣⍺⍺}
r←⎕NS⍬
:for sect :in clou⊂⍨~'='∊⍤0 1⊢↑clou
   :if ''≡⊃sect ⋄ :continue ⋄ :endif
   sectns←⎕NS⍬
   (sectns _set)/↑'([^= ]+)\s*=\s*(.+)'⎕s{⍵.(1↓Lengths↑¨Offsets↓¨⊂Block)}1↓sect
   (1↓¯1↓⊃sect)(r _set)sectns
:endfor
∇

In [10]:
clou2←{⎕JSON⍠'Dialect' 'JSON5'⊢2⌽ '}}{"":{' , ∊'^;.*' '^\[(.*)]' '^(.*?)\s*=\s*(.*)'⎕R'' '},"\1":{' '"\1":"\2",' ⊢ ⊃⎕NGET ⍵}

## More grouping: left join


Can anyone suggest a clever way of achieving the follwing "left join":

Given two vectors, the first a vector of char vecs, the second a vector of two-element vectors, group the second based on the first component, returning the second components per group, inserting the empty vectors for groups not present.
e.g

```apl
X←'one' 'two' 'three' 'four'
Y←('one' 1)('one' 2)('three' 1)('four' 2)('four' 3)
R←⎕←X f Y
┌───┬┬─┬───┐
│1 2││1│2 3│
└───┴┴─┴───┘
R≡(1 2)(,⍬)(,1)(2 3)
1
```

In [7]:
X←'one' 'two' 'three' 'four'
Y←('one' 1)('one' 2)('three' 1)('four' 2)('four' 3)
(⊃¨{⊂1↓⊢/¨⍵}⌸⊢)(X,⍥⊂¨0),Y ⍝ Suggested by Adam

┌───┬┬─┬───┐
│1 2││1│2 3│
└───┴┴─┴───┘

One from @ngn

In [8]:
K V←↓⍉↑Y ⋄ R←⍬⍨¨X ⋄ R[X⍳K],←V
R

┌───┬┬─┬───┐
│1 2││1│2 3│
└───┴┴─┴───┘

## Times and dates

See https://chat.stackexchange.com/rooms/52405/conversation/lesson-48-c-dt-1200

File last modification time, epoch seconds:

In [11]:
20 ⎕DT 3⎕NINFO '/Users/stefan/user.json'

1593593640

Current time, epoch seconds

In [12]:
20 ⎕DT ⊂⎕TS

1615549666

In [ ]:
md5←{
     ⍝ By LdBeth (https://math.stackexchange.com/users/899178/ldbeth)
     ⍝ index origin zero
     ⎕IO←0
     ⍝ decoding UTF-8 & padding
     M←(⊢,(0⍴⍨512|448-512|≢))1,⍨l←,⍉(8⍴2)⊤'UTF-8'⎕UCS ⍵
     ⍝ add length
     M,←,⍉(8⍴2)⊤⌽(8⍴256)⊤≢l
     ⍝ init registers (@ngn suggests A B C D←16⊥⍉(⌽⍪⊖)⍪⌽2 4 2⍴⍳16)
     A←16⊥6 7 4 5 2 3 0 1
     B←16⊥14 15 12 13 10 11 8 9
     C←16⊥9 8 11 10 13 12 15 14
     D←16⊥1 0 3 2 5 4 7 6
     ⍝ 
     ⍝ T table
     T←⌊(2*32)×|1○1+⍳64
     ⍝ index table
     K←16|i,(1+5×i),(5+3×i),7×i←⍳16
     ⍝ rot table
     S←,1 0 2⍉4 4 4⍴7 12 17 22 5 9 14 20 4 11 16 23 6 10 15 21
     ⍝ truncate ⍵ to 32 bit & rot left ⍺
     rot←{2⊥⍺⌽(32⍴2)⊤⍵}
     proc←{
         ⍝ pack 512 bits into 32 bit words &
         ⍝ precompute X[k] + T[i]
         l←T+(⊂K)⌷256⊥⍉⌽16 4⍴2⊥⍉64 8⍴⍺
         fn←{
         ⍝ a b c d to binary
             a b c d←↓⍉(32⍴2)⊤⍵
         ⍝ a + F(b,c,d)
             ⍺<16:S[⍺]rot l[⍺]+2⊥a+d≠b∧c≠d
             ⍺<32:S[⍺]rot l[⍺]+2⊥a+(b∧d)∨(c∧~d)
             ⍺<48:S[⍺]rot l[⍺]+2⊥a+b≠c≠d
             S[⍺]rot l[⍺]+2⊥a+c≠b∨~d
         }
         (2*32)|⍵+⊃{¯1⌽((⍵[1]+⍺ fn ⍵)@0)⍵}/(⌽⍳64),⊂⍵
     }
     ⍝ process each 512 bits
     loop←{⍬≡⍺:⍵ ⋄ (512↓⍺)∇(512↑⍺)proc ⍵}
     ⍝ output registers
     (⎕D,⎕A)[,⍉(2⍴16)⊤,⍉⊖(4⍴256)⊤M loop A B C D]
 }

In [ ]:
]dinput
Merge←{
    templ←⎕JSON (⊃⎕NGET ⍺)
    keys←templ.⎕NL¯2
    vals←templ.(⍎¨#.keys)
    
    (({'@',(⍕⍵),'@'}¨keys),(⊂'@@'),⊂'@[^@]+@')⎕R((⍕¨vals),(⊂,'@'),⊂'???')⊢⊃⎕NGET ⍵
}

## Commandline args

In [3]:
⊢2⎕NQ#'GetCommandLineArgs'

┌──────────────────────────────────────────────────────────────┬──┬──┬─────────────────────────────────────────────────────────────┐
│/Applications/Dyalog-18.1.app/Contents/Resources/Dyalog/dyalog│+s│-q│/opt/anaconda3/envs/py39/lib/python3.9/dyalog_kernel/init.dws│
└──────────────────────────────────────────────────────────────┴──┴──┴─────────────────────────────────────────────────────────────┘

## Partition Problem of Equal Sums

https://en.wikipedia.org/wiki/Partition_problem

In [ ]:
]dinput
HorowitzSahni ← {
    sum←1⊥⍵
    2|sum: ⍬   ⍝ Lists with an odd sum cannot be split into equal parts.
    halfsum←sum÷2
    s←⍵(↑{⍺⍵}↓)⍨⌊2÷⍨≢⍵                          ⍝ Split the input.
    a b←⊃¨(⊢,+)/¨s,¨0                           ⍝ Generate the subset sums.
    indexes←a {(⊢,⍵⍳⍺⌷⍨(≢⍺)⌊⊢)1⍳⍨⍺∊⍵} halfsum-b ⍝ Search for solution indexes.
    indexes[2]>≢b: ⍬
    ⍵ {(⍺/⍨~⍵)(⍵/⍺)} ∊(2⍴¨⍨≢¨s)⊤¨indexes-1      ⍝ Get the solution from the indexes.
}

## eXplanation operator for products

https://chat.stackexchange.com/transcript/52405?m=43945029#43945029


In [3]:
X←{f←⍺⍺ ⋄ ⍺←⊢ ⋄ '(',⍺,(⎕CR'f'),⍵,')'}
(2 2⍴⍳4) +.× 2 2⍴10+⍳4
(2 2⍴⍳4) +X.(×X) 2 2⍴10+⍳4
'abc'(+X).(×X)'DEF'

37 40
85 92

┌───────────────────────┬───────────────────────┐
│(( 1 × 11 )+( 2 × 13 ))│(( 1 × 12 )+( 2 × 14 ))│
├───────────────────────┼───────────────────────┤
│(( 3 × 11 )+( 4 × 13 ))│(( 3 × 12 )+( 4 × 14 ))│
└───────────────────────┴───────────────────────┘

┌─────────────────────┐
│((a×D)+((b×E)+(c×F)))│
└─────────────────────┘

## Complex numbers as points/grids

In [ ]:
0j1⊥¨    0j1⊥   ⍝ pair(s) of reals -> complex
11 9∘○¨  11 9○  ⍝ complex -> pair(s) of reals
|z0-z1          ⍝ distance between two points
0j1×z   0j¯1×z  ⍝ rotate by ±90° around (0,0)
0j1*⍳4          ⍝ the four cardinal directions
+z       -+z    ⍝ reflect across x or y axis
+\0,z           ⍝ sequence of steps -> path
¯2-/z           ⍝ path -> sequence of steps
0j1⊥¨n-⍳2⍴1+2×n ⍝ lattice centred on (0,0)

## Longest stretch of 1s

In [2]:
ones←{≢⍉↑⊆⍨⍵} ⍝ Slower, but cool (@ngn)
onesf←⌈/(0~⍨¯1-2-/∘⍸1,1,⍨~) ⍝ Faster (@adam)

In [4]:
ones 1 1 0 1 1 1 0 1 1 1 1 1 1 0 1 0 1 0
onesf 1 1 0 1 1 1 0 1 1 1 1 1 1 0 1 0 1 0

6

6

## RepObj

Display any object as an executable constructor

In [5]:
⎕SE.Dyalog.Utils.repObj 2 3⍴6?6

2 3⍴4 1 6 5 2 3

In v18.1, this is available as `]repr`

In [2]:
]repr 2 3⍴6?6

2 3⍴4 1 6 5 2 3

## Longest Increasing Subsequence (nlogn)
Direct re-implementation of wikpedia pseudocode. ⎕IO←0

In [1]:
∇ S←lis X;N;P;M;L;i;lo;hi;mid;newL;k
⍝ https://en.wikipedia.org/wiki/Longest_increasing_subsequence
N←≢X
P←N↑¯1
M←(N+1)↑¯1
L←0
:for i :in ⍳N
    (lo hi)←1 L
    :while lo≤hi
        mid←⌈(lo+hi)÷2
        :if X[M[mid]]<X[i]
            lo←mid+1
        :else
            hi←mid-1
        :endif
    :endwhile
    newL←lo
    P[i]←M[newL-1]
    M[newL]←i
    :if newL>L
        L←newL
    :endif
:endfor
S←L↑0
k←M[L]
:for i :in ⊖⍳L
    S[i]←X[k]
    k←P[k]
:endfor
∇

In [1]:
glue2←{m←⍸(⌽¨,\⌽⍺)≡¨(,\⍵) ⋄ m≡⍬:⍬ ⋄ ⍺,m↓⍵}
glue3←{⍺,⍵↓⍨⌈/0,⍸(↑∘⍵≡-↑⍺⍨)¨⍳⍺⌊⍥≢⍵}
glue4←'(.*);\1'⎕R'\1'⊣,';',⊢

## Longest common sub-sequence

https://raw.githubusercontent.com/Masterquiz/APL-Problem-Solving-Competition/main/2021/Phase2/Problem7/lcsq.aplf

In [1]:
]dinput
lcsq ← {
    M←↑⊃{⍵,⊂⌈\last⌈(⍺+0,¯1↓last←⊃⌽⍵)⌈0,¯1↓⍺}/(⊂⊂{0}¨⍵),⍨⌽↓⍺∘.=⍵
    mat←2<⌿2</0,M
    pos←(⍸mat){⍵[⍋⍵]⊆⍺[⍋⍵]}mat/⍥,1↓M
    ind←1↓¨{⍵,¯1↑⍺/⍨∧/¨⍺<¯1↑⍵}/pos,⊂⊂⍴M
    ⌽⍺⌷⍨⊃¨¨ind
}

In [2]:
assert←{⍺←'assertion failure' ⋄ 0∊⍵:⍺ ⎕signal 8 ⋄ shy←0}
assert 'AACTTG' ≡ 'AACCTTGG' lcsq 'ACACTGTGA'
assert '' ≡ 'ACGTACGTGACG' lcsq ''
assert '' ≡ 'ACGTACGTGACG' lcsq 'XYZZYX'

## Boolean alternating selection

Given two arrays A, B and a boolean filter C, select items from A if C is false and from B if C is true

In [4]:
A←1 2 3 4
B←11 22 33 44
C←0 0 1 1
(C/⍥,B)@{C}A

1 2 33 44

If we're dealing with vectors, that's just

In [3]:
(C/B)@{C}A

1 2 33 44

All that says is: replace the true spots (as defined by C) in A with the true spots (as defined by C) from B. If we can mutate A, we could also have written that as

In [5]:
(C/A)←C/B
A

1 2 33 44

## Hashmap from dzaima

In [1]:
]dinput
map←{
  r←⎕NS ⍬
  r.k←1500⌶⍬
  r.v←⍬
  r.set←{
    i←k⍳⊂⍺
    i≡1+≢k: v,←⊂⍵⊣k,←⊂⍺
    v[i]←⊂⍵
  }
  r.get←{
    (k⍳⊂⍵)⊃v
  }
  r
}

## Performance gotchas

In [1]:
a ← ?⍨ 100000
]runtime -c '⊃⍤⍋a' '⊃⍋a'

⊃⍤⍋a → 8.7E¯6 | 0% ⎕⎕ 
 ⊃⍋a → 1.5E¯4 | +1647% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕

## Regex primes

In [10]:
IsPrime←{⍬≡'^.?$|^(..+?)\1+$'⎕S 3⊢'1'/⍨⍵}

In [11]:
nums/⍨IsPrime¨nums←2 5 6 7 8 9 10 11

2 5 7 11

## Binary search

In [1]:
]dinput
bsearch←{⎕IO←0
   0 ((,⍺) {
       ⍵<⍺:⍬
       m←⌊2÷⍨⍺+⍵
       ⍺⍺[m]>⍵⍵:⍺∇m-1
       ⍺⍺[m]<⍵⍵:⍵∇⍨m+1
       m
   } ⍵)¯1+≢,⍺                
}

In [3]:
]display 1 2 3 5 7 8 9 12 bsearch 11
1 2 3 5 7 8 9 12 bsearch 5
1 bsearch 1

┌⊖┐
│0│
└~┘

3

0

## Fibobnacci sequence

See https://www.jsoftware.com/papers/50/50_36.htm

In [5]:
fib←{r←⍬⋄_←{{p←+\⌽⍵⋄r,←⊃p⋄p}⍣⍵⊢0 1}⍵⋄r}

In [6]:
fib 10

1 1 2 3 5 8 13 21 34 55

In [1]:
fib2←{s÷⍨(p*⍵)-⍵*⍨1-p←2÷¯1+s←5*÷2}⍳ ⍝ Via Binet's formula. Likely the fastest Fib

In [2]:
fib2 10

1 1 2 3 5 8 13 21 34 55

## Farey sequence

https://en.wikipedia.org/wiki/Farey_sequence

In [11]:
]dinput
farey ← {⎕IO←0
    r ← ⊂0 1
    0=⍵: ,r
    n ← ⍵
    _ ← {
        (a b c d) ← ⍵
        k ← ⌊d÷⍨n+b
        r ,← ⊂c d
        c d,(k×c d)-a b
    }⍣{n<2⊃⍺} 0 1 1 n
    r
}

In [12]:
farey 5

┌───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┐
│0 1│1 5│1 4│1 3│2 5│1 2│3 5│2 3│3 4│4 5│1 1│
└───┴───┴───┴───┴───┴───┴───┴───┴───┴───┴───┘

## Progressive Dyadic Iota

https://dyalog.tv/Webinar/?v=6SAsgEvUmkU

In [7]:
pdi ← {((⍴⍺)⍴⍋⍋⍺,⍵)⍳(⍴⍵)⍴⍋⍋⍵,⍺}

In [8]:
'DYALOG APL' pdi 'AAALLLB'

3 8 11 4 10 11 11

## Keep non-duplicated major cells


In [15]:
(∪(⌿⍨)1=⊢∘≢⌸)'abcdghidcba'

ghi

## Show all glyphs the interpreter recognises

In [16]:
a/⍨{'.'=⍵:1⋄0::0⋄⊃⌽19=4↑∊200⌶⎕NR ⎕FX'f'⍵}¨a←⎕ucs⍳1e4

!&*+,-./<=>?@\^|~¨×÷←↑→↓∊∘∣∧∨∩∪≠≡≢≤≥⊂⊃⊆⊖⊢⊣⊤⊥⌈⌊⌶⌷⌸⌹⌺⌽⌿⍀⍉⍋⍎⍒⍕⍟⍠⍣⍤⍥⍨⍪⍬⍱⍲⍳⍴⍷⍸○

Better:

In [1]:
⎕ucs 2031⌶6 

+-×÷⌈⌊*⍟|!○~∨∧⍱⍲<≤=≥>≠.@≡≢⍴,⍪⍳↑↓?⍒⍋⍉⌽⊖∊⊥⊤⍎⍕⌹⊂⊃∪∩⍷⌷∘→←⎕⍞/⌿\⍀¨⍣&⍨⌶#⊆⍥⊣⊢⍠⍤⌸⌺⍸()[];
 ⍝⋄:⍬{}∇

Missing only `⍺⍵`.

## More on zip

This is clever use of grade [APLCart](https://aplcart.info/?q=Merge%20X%20and%20Y%20alternately#)

In [17]:
'short' {(⊂⍋∊⍳∘≢¨⍺ ⍵)⌷⍺⍪⍵} 'much longer' ⍝ Merge X and Y alternately

smhuocrht longer

## (Too) Simple APL grammar

    atom ::= id | integer-literal | '(' expr ')'
    obj  ::= atom | atom obj
    fn   ::= atom | '{' expr '}' | obj mop | fn dop obj | obj dop obj
    expr ::= obj | atom fn expr | fn expr

## Identity matrix generator

In [1]:
{⍵ ⍵⍴(1+⍵)↑1}4

1 0 0 0
0 1 0 0
0 0 1 0
0 0 0 1

In [2]:
(,⍨⍴1↑⍨1∘+)4

1 0 0 0
0 1 0 0
0 0 1 0
0 0 0 1

## Flood fill with stencil

In [1]:
(××{1⊃⌈/⍵,⍉⍵}⌺3 3)⍣≡ ⍝ max-of-neighbours flood fill of non-zero cells

## Dijkstra by @dzaima

https://adventofcode.com/2021/day/15

This is an implementation of Dijkstra's algorithm for a square grid of weights. Every node is connected to four of its neighbors. 

ll - flat vector of the costs; dirs - lists representing all neighbors for each item in ll; w - width (also height); sc - current "score" of a cell; c - the list of indices that need to be updated in the next step.

(sc has a ≢llth element that's always the maximum value, and dirs references that for the things that'd otherwise go out-of-bounds)

A similar solution in Python: https://github.com/hyper-neutrino/advent-of-code/blob/main/2021/day15p2.py

In [3]:
⎕IO←0
l←↑48-⍨⎕ucs¨⊃⎕NGET'../d/15'1
ll w sc c dirs ← ⍬⍬⍬⍬⍬ 

In [4]:
]dinput
init ← {
    w⊢← ≢⍵                       ⍝ dimensions (maze is square)
    ll⊢← ,⍵                      ⍝ flat vector of the costs
    tl ← ≢ll
    sc⊢← 0@0⊢ (1+tl) ⍴ 10×+/ll   ⍝ current "score" of a cell
    c⊢← 1 w                      ⍝ list of indices that need to be updated in the next step. 1 and w
                                 ⍝ are the two 4-connected neighbours of node 0.
                                 
    dirs⊢← ,¨ {(tl⍪⍨1↓⍵)(tl⍪¯1↓⍵)(tl,0 ¯1↓⍵)(tl,⍨0 1↓⍵)} w w⍴⍳tl ⍝ Build the neighbours map
}

In [5]:
]dinput
step ← {                             ⍝ Dijkstra cheapest path
    p ← sc[c]                        ⍝ previous score for each current cell
    nd ← {⍵[c]}¨ dirs                ⍝ neighbor indexes
    n ← ll[c] + ⊃⌊/ {sc[⍵]}¨ nd      ⍝ new best score for the cell: my current total plus the cheapest neighbor
    m ← n<p                          ⍝ mask of improved scores
    sc[m/c] ← m/n                    ⍝ update cell scores of improvers
    c⊢← ∪ ⊃,/ {(m ∧ (≢ll)≠⍵)/⍵}¨ nd  ⍝ indices that need to be updated in the next step, removing dupes
    c
}

In [6]:
init l
_←step⍣{0=≢⍺} c
1⊃⌽sc

init 1+9|¯1+ ⊃⍪/,/(⊂l) + ⊃¨ +/¨⍳5 5
_←step⍣{0=≢⍺} c
1⊃⌽sc

537

2881

## Maze to connectivity vector (graph)

In [11]:
⎕IO←0
'path'⎕CY'dfns'
⊢m←(5 13⍴(14/0),(11/1),(4/0),1 0 1 0 1 0 1,(6/0),1 0 1 0 1 0 1,16/0)
shifts←{(⍺,2⊣/⍵)(2⊢/⍵,⍺)(2⊢⌿⍵⍪⍺)(⍺⍪2⊣⌿⍵)} ⍝ LEFT RIGHT DOWN UP
abspos←{⍺/(⊂⍵)+(0 ¯1)(0 1)(1 0)(¯1 0)}
n4←2 0 1⍉↑0 shifts m

0 0 0 0 0 0 0 0 0 0 0 0 0
0 1 1 1 1 1 1 1 1 1 1 1 0
0 0 0 1 0 1 0 1 0 1 0 0 0
0 0 0 1 0 1 0 1 0 1 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0

In [12]:
nodes←⍸m
vec←{(⍵⌷n4)abspos ⍵}¨nodes
⊢graph←nodes∘⍳¨vec

┌─┬───┬──────┬───┬──────┬───┬──────┬───┬──────┬────┬─┬────┬────┬────┬────┬──┬──┬──┬──┐
│1│0 2│1 3 11│2 4│3 5 12│4 6│5 7 13│6 8│7 9 14│8 10│9│15 2│16 4│17 6│18 8│11│12│13│14│
└─┴───┴──────┴───┴──────┴───┴──────┴───┴──────┴────┴─┴────┴────┴────┴────┴──┴──┴──┴──┘

In [14]:
nodes[graph path 0 18]

┌───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┐
│1 1│1 2│1 3│1 4│1 5│1 6│1 7│1 8│1 9│2 9│3 9│
└───┴───┴───┴───┴───┴───┴───┴───┴───┴───┴───┘

## Leap year

Genius: https://codegolf.stackexchange.com/a/50805/94243

In [5]:
{~≥/⌽×4 25 4⊤⍵} 2000

1

APL Cart offers the following:

In [6]:
(0≠.=400 100 4∘.|⊢)2000

1

## Convert Boolean vector to string, without spaces

In [9]:
⎕UCS 48+0 1 1 0 0 1 0 1 0 0 1 1 0 1  ⍝ Fastest
∊⍕¨0 1 1 0 0 1 0 1 0 0 1 1 0 1       ⍝ Shortest
⍕10⊥0 1 1 0 0 1 0 1 0 0 1 1 0 1      ⍝ Tempting, but will fail for longer strings due to numeric precision

01100101001101

01100101001101

1.1001E12

## Substring replace

Here's a dyadic operator deriving a monadic function that replaces each occurrence of a substring with a replacement string, returning all intermediate states:

In [12]:
⎕IO←0
_ss_←{⍵⍵∘(⍺⍺{(⍵↑⍵⍵),⍺,(⍵⍵↓⍨⍵+≢⍺⍺)}⍵)¨⍸⍺⍺⍷⍵}

In [14]:
('ab' _ss_ 'ABC')'ab ab ab'

┌─────────┬─────────┬─────────┐
│ABC ab ab│ab ABC ab│ab ab ABC│
└─────────┴─────────┴─────────┘

## Power scan

In k you can do

```
 {_x%2}\32

32 16 8 4 2 1 0
```
but in APL we have to resort to

In [15]:
{⌊⍵÷2}{⍺←⊢ ⋄ r⊣⍺ ⍺⍺{⍺←⊢ ⋄ r,∘⊂←⍺ ⍺⍺ ⍵}⍣⍵⍵⊃r←⊂⍵}{⍺=0}32

32 16 8 4 2 1 0

## Longest palindromic substring

This is Mannacher's algorithm, at O(n). Very clever, and fast.

In [18]:
∇ res←manacher str;S;c;r;radii;i;ir
c r radii←0 0 (0↑⍨≢S←'|',∊'|',⍨⍪str)

:for i :in ⍳≢S
    radii[i]←{r>i:radii[(2×c)-i]⌊r-i⋄0}⍬
    :trap 3 ⍝ Catch index errors if the below goes out of bounds
        :while =/S[i(+,-)1+i⊃radii]
            radii[i]+←1
        :endwhile
    :endtrap
    :if r<ir←i+i⊃radii
        c r←i ir
    :endif
:endfor

c←radii⍳r←⌈/radii
res←S[c(+(⊢+∘⍳1+-)-)r]~'|'
∇

## Median

In [19]:
{2÷⍨+⌿⍵[(⍋⍵)[⌈2÷⍨0 1+≢⍵]]} 1 8 5 2 3 9

6.5

## Set first and last element of a boolean vector to 1

In [2]:
⎕IO←0
v←0 1 0 1 1 0 1 0 1
1@0⍤⌽⍣2⊢v

v←0 1 0 1 1 0 1 0 1
(2↑¯1⌽v)←1              ⍝ Mutating
v

v←1 1 0 1 1 0 1 0 0
(⍷⍨∨⌽)⍣2⊢v              
(⍷⍨∨⌽)⍣2⊢⍬     ⍝ Note: this also works for empty array

(⊢∨∨∘⌽⍨⍤⍷⍨)v

(⍷∨⊢∨⌽⍤⍷)⍨v

(⍷⍨∨⊢∨⌽⍤⍷⍨)v


⍝ Set last only
v←0 1 0 1 1 0 1 0 0
⌽1@0⊢⌽v

1 1 0 1 1 0 1 0 1

1 1 0 1 1 0 1 0 1

1 1 0 1 1 0 1 0 1

1 1 0 1 1 0 1 0 1

1 1 0 1 1 0 1 0 1

1 1 0 1 1 0 1 0 1

0 1 0 1 1 0 1 0 1

## Turn off first 1 in Boolean vector

In [3]:
(⊢><\)0 0 0 1 0 0 1 0 1

0 0 0 0 0 0 1 0 1

## Split number into integer and fractional parts

In [1]:
0 1⊤1.234

Rebuilding user command cache... 
done

1 0.234

## Partition on inverse where
This is pretty awesome:

In [7]:
⎕IO←0
(⍸⍣¯1⊢0 3 4)⊂'KenEIverson'

┌───┬─┬───────┐
│Ken│E│Iverson│
└───┴─┴───────┘

## Longest unbroken sequence of 1s in a boolean vector

https://aplcart.info/?q=lengths%20runs%201s#

In [3]:
(⌈/¯1-2-/∘⍸1,1,⍨~) 1 1 1 0 0 1 1 1 1 0 0 1 1 0 1 1           ⍝ Flat, fast, but complex
(⌈/≢¨⍤⊆⍨) 1 1 1 0 0 1 1 1 1 0 0 1 1 0 1 1                    ⍝ Nested, VERY slow, but clear
{¯1-(⊃i)⌊⌊/2-/i←(⍸~⍵),1+≢⍵} 1 1 1 0 0 1 1 1 1 0 0 1 1 0 1 1  ⍝ Flat, fastest

4

4

4

## Modulo exponentiation

Via APLCart:

In [4]:
2 (1000000{⍺⍺{⍺⍺|⍺×⍺⍺|×⍨⍵}⌿⍺*⍤1⊖0⍪2⊥⍣¯1⊢⍵}) 480 ⍝ 1E6|2*480

290176

## CSV hacks

Convert a string to an APL vector, converting any numbers. The data description column given as 'S' for Simple. Default is 'N' for Nested.

In [23]:
{(,⎕CSV⍠'Separator' ' ') ⍵ 'S' 4} '23 -76.2 87 "hello" -1 0.6'

┌──┬─────┬──┬─────┬──┬───┐
│23│¯76.2│87│hello│¯1│0.6│
└──┴─────┴──┴─────┴──┴───┘

Read a file containing one number per line into an APL vector:

In [1]:
{,⎕CSV ⍵ ⍬ 2} 'data/nums.txt'

123 ¯34 3.1415 54 76

Read a spread-sheety CSV-file, converting numbers:

In [24]:
{⎕CSV ⍵ ⍬ 4} 'data/Sample-Spreadsheet-100-rows.csv'

┌───┬───────────────────────────────────────────────────────────────────────────────────────────────┬──────────────────┬─────┬────────┬───────┬─────┬─────────────────────┬──────────────────────────────┬────┐
│1 │Eldon Base for stackable storage shelf, platinum │Muhammed MacIntyre│3 │¯213.25 │38.94 │35 │Nunavut │Storage & Organization │0.8 │
├───┼───────────────────────────────────────────────────────────────────────────────────────────────┼──────────────────┼─────┼────────┼───────┼─────┼─────────────────────┼──────────────────────────────┼────┤
│2 │1.7 Cubic Foot Compact "Cube" Office Refrigerators │Barry French │293 │457.81 │208.16 │68.02│Nunavut │Appliances │0.58│
├───┼───────────────────────────────────────────────────────────────────────────────────────────────┼──────────────────┼─────┼────────┼───────┼─────┼─────────────────────┼──────────────────────────────┼────┤
│3 │Cardinal Slant-D® Ring Binder, Heavy Gauge Vinyl │Barry French │293 │46.71 │8.69 │2.99 │Nunavut │Binders and Binder Accessories│0.39│
├───┼───────────────────────────────────────────────────────────────────────────────────────────────┼──────────────────┼─────┼────────┼───────┼─────┼─────────────────────┼──────────────────────────────┼────┤
│4 │R380 │Clay Rozendal │483 │1198.97 │195.99 │3.99 │Nunavut │Telephones and Communication │0.58│
├───┼───────────────────────────────────────────────────────────────────────────────────────────────┼──────────────────┼─────┼────────┼───────┼─────┼─────────────────────┼──────────────────────────────┼────┤
│5 │Holmes HEPA Air Purifier │Carlos Soltero │515 │30.94 │21.78 │5.94 │Nunavut │Appliances │0.5 │
├───┼───────────────────────────────────────────────────────────────────────────────────────────────┼──────────────────┼─────┼────────┼───────┼─────┼─────────────────────┼──────────────────────────────┼────┤
│6 │G.E. Longer-Life Indoor Recessed Floodlight Bulbs │Carlos Soltero │515 │4.43 │6.64 │4.95 │Nunavut │Office Furnishings │0.37│
├───┼───────────────────────────────────────────────────────────────────────────────────────────────┼──────────────────┼─────┼────────┼───────┼─────┼─────────────────────┼──────────────────────────────┼────┤
│7 │Angle-D Binders with Locking Rings, Label Holders │Carl Jackson │613 │¯54.04 │7.3 │7.72 │Nunavut │Binders and Binder Accessories│0.38│
├───┼───────────────────────────────────────────────────────────────────────────────────────────────┼──────────────────┼─────┼────────┼───────┼─────┼─────────────────────┼──────────────────────────────┼────┤
│8 │SAFCO Mobile Desk Side File, Wire Frame │Carl Jackson │613 │127.7 │42.76 │6.22 │Nunavut │Storage & Organization │ │
├───┼───────────────────────────────────────────────────────────────────────────────────────────────┼──────────────────┼─────┼────────┼───────┼─────┼─────────────────────┼──────────────────────────────┼────┤
│9 │SAFCO Commercial Wire Shelving, Black │Monica Federle │643 │¯695.26 │138.14 │35 │Nunavut │Storage & Organization │ │
├───┼───────────────────────────────────────────────────────────────────────────────────────────────┼──────────────────┼─────┼────────┼───────┼─────┼─────────────────────┼──────────────────────────────┼────┤
│10 │Xerox 198 │Dorothy Badders │678 │¯226.36 │4.98 │8.33 │Nunavut │Paper │0.38│
├───┼───────────────────────────────────────────────────────────────────────────────────────────────┼──────────────────┼─────┼────────┼───────┼─────┼─────────────────────┼──────────────────────────────┼────┤
│11 │Xerox 1980 │Neola Schneider │807 │¯166.85 │4.28 │6.18 │Nunavut │Paper │0.4 │
├───┼───────────────────────────────────────────────────────────────────────────────────────────────┼──────────────────┼─────┼────────┼───────┼─────┼─────────────────────┼──────────────────────────────┼────┤
│12 │Advantus Map Pennant Flags and Round Head Tacks │Neola Schneider │807 │¯14.33 │3.95 │2 │Nunavut │Rubber Bands │0.53│
├───┼───────────────────────────────────────────────────────────────────────────────────────────────┼────────────────

## Huge k idioms list
https://web.archive.org/web/20071230205056/http://kx.com/technical/contribs/eugene/kidioms.html

## Odometer

The obvious multivariate odometer is

In [7]:
⎕IO←0
,⍳3 5

┌───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┐
│0 0│0 1│0 2│0 3│0 4│1 0│1 1│1 2│1 3│1 4│2 0│2 1│2 2│2 3│2 4│
└───┴───┴───┴───┴───┴───┴───┴───┴───┴───┴───┴───┴───┴───┴───┘

but it suffers from being both slow and topping out at 16 dimensions (unlikely to present a problem in run of the mill applications).

A different way of approaching this is to use _encode_:

In [9]:
⎕IO←0
⊂[0]3 5⊤⍳3×5

┌───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┐
│0 0│0 1│0 2│0 3│0 4│1 0│1 1│1 2│1 3│1 4│2 0│2 1│2 2│2 3│2 4│
└───┴───┴───┴───┴───┴───┴───┴───┴───┴───┴───┴───┴───┴───┴───┘

In [10]:
]dinput
odo ← {
    ⊂[0]⍵⊤⍳×/⍵
}

In [11]:
odo 3 5

┌───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┐
│0 0│0 1│0 2│0 3│0 4│1 0│1 1│1 2│1 3│1 4│2 0│2 1│2 2│2 3│2 4│
└───┴───┴───┴───┴───┴───┴───┴───┴───┴───┴───┴───┴───┴───┴───┘

## Spiral

Algorithm due to Graham, Knuth, and Patashnik.

https://www.jsoftware.com/papers/play132.htm

https://dl.acm.org/doi/pdf/10.1145/28315.28370#page=6

In [12]:
⎕IO←1
{(2⍴⍵)∘⍴⍋+⍀∘∊(0,⊢×((≢⍴1 1 ¯1 ¯1⍨)×(≢⍴(1,⍵)⍨)))(⍸,)¨⍸3,⍨(⍵-2)⍴2}5

21 22 23 24 25
20 7 8 9 10
19 6 1 2 11
18 5 4 3 12
17 16 15 14 13

Here's a different one, going the other way:

In [13]:
{⍵ ⍵⍴⍋(⍳⍵*2)[+\a/(≢a←1↓⌽2/⍳⍵)⍴,∘-⍨1⍵]}5

1 2 3 4 5
16 17 18 19 6
15 24 25 20 7
14 23 22 21 8
13 12 11 10 9

## Boolean vector twiddles

Set all bits up to, and including, the first set bit, unset the rest:

In [1]:
⊢a←0 1 0 1
(<\∨∧\⍤~)a

0 1 0 1

1 1 0 0

An alterntive, shorter formulation:

In [2]:
(∨\≤<\)a

1 1 0 0

## Efficient iota with vector argument

Often you don't care about the shape of `⍳a b c ...` but only want the ravel data. Moreover, the iota is subject to Dyalog's max rank. Here's another way (note: only `⎕IO←0`, and note ordering not the same):

In [8]:
↓⍉(⊢⊤∘⍳×/) 3 3 3

┌─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬
│0 0 1│0 0 2│0 1 0│0 1 1│0 1 2│0 2 0│0 2 1│0 2 2│1 0 0│1 0 1│1 0 2│1 1 0│1 1 1│
└─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴
 ─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─
 1 1 2│1 2 0│1 2 1│1 2 2│2 0 0│2 0 1│2 0 2│2 1 0│2 1 1│2 1 2│2 2 0│2 2 1│2
 ─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─
 ────┬─────┐
 2 2│0 0 0│
 ────┴─────┘

In [9]:
↓⍉3 3 3⊤⍳×/3 3 3 ⍝ Explicit form

┌─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬
│0 0 1│0 0 2│0 1 0│0 1 1│0 1 2│0 2 0│0 2 1│0 2 2│1 0 0│1 0 1│1 0 2│1 1 0│1 1 1│
└─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴
 ─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─
 1 1 2│1 2 0│1 2 1│1 2 2│2 0 0│2 0 1│2 0 2│2 1 0│2 1 1│2 1 2│2 2 0│2 2 1│2
 ─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─
 ────┬─────┐
 2 2│0 0 0│
 ────┴─────┘

## Arbitrarily merge two strings, given pattern

In [1]:
(⊂⍋⍋0 0 0 1 1 1 0 0 0)⌷'fooxyz','qwe'

fooqwexyz

In [2]:
3(↑,'qwe',↓)'fooxyz'

fooqwexyz

In [3]:
{0::0 ⋄ ≢⎕NEW⊂'HTMLRenderer'}⍬

0